In [1]:
!pip install scispacy spacy pandas transformers
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_lg-0.5.1.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.2/920.2 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: blis
    Found existing installation: blis 1.3.0
    Uninstalling blis-1.3.0:
      Successfully uninstalled blis-1.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.3/532.3 MB 3.4 MB/s eta 0:00:00
ERROR: Operation cancelled by user
^C


In [1]:
!pip install scispacy spacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bionlp13cg_md-0.5.1.tar.gz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 MB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.4/917.4 kB 48.3 MB/s eta 0:00:00
  Created wheel for en_ner_bionlp13cg_md: filename=en_ner_bionlp13cg_md-0.5.1-py3-none-any.whl size=120241137 sha256=6a7df33e50a3d82cc264c3d7b82a6b559c1bce5646c130307483adeb0dc6b544
  Stored in directory: /root/.cache/pip/wheels/1b/48/06/cb86feaf8cf8bb0d00a9465e788e3f19cc81e931c6a69b9859
Successfully built en_ner_bionlp13cg_md
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.3


In [2]:
import spacy
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

nlp = spacy.load("en_ner_bionlp13cg_md")

In [3]:
import pandas as pd

df = pd.read_excel('merged_sentences_with_meta.xlsx', engine='openpyxl')
df = df.drop_duplicates(subset='Sentence', keep='first').reset_index(drop=True)


In [4]:
import numpy as np
import pandas as pd
import pickle
import hashlib
import sqlite3
import tempfile
from collections import defaultdict
from numba import jit
from multiprocessing import Pool, cpu_count
import time
import spacy
import networkx as nx

nlp = spacy.load("en_ner_bionlp13cg_md", disable=["parser", "tagger", "lemmatizer"])

def process_chunk_parallel(args):
    sentences, nlp_model_name = args
    import spacy
    nlp = spacy.load(nlp_model_name, exclude=["parser", "tagger", "lemmatizer"])

    results = []
    docs = list(nlp.pipe(sentences, disable=['parser', 'tagger', 'lemmatizer']))
    for sent, doc in zip(sentences, docs):
        ents = [ent.text.strip().lower() for ent in doc.ents if len(ent.text.strip()) > 2 and not ent.text.strip().isdigit()]
        results.append((sent, ents))
    return results

def parallel_processing_version(df, nlp_model_name='en_ner_bionlp13cg_md', n_processes=None):
    if n_processes is None:
        n_processes = min(cpu_count(), 4)

    sentences = df['Sentence'].dropna().astype(str).str.strip().str.lower().tolist()
    chunked = np.array_split(sentences, n_processes)
    chunk_args = [(list(chunk), nlp_model_name) for chunk in chunked]

    with Pool(n_processes) as pool:
        results = pool.map(process_chunk_parallel, chunk_args)

    all_results = []
    for r in results:
        all_results.extend(r)
    return all_results

@jit(nopython=True)
def fast_combinations(n):
    return n * (n - 1) // 2

def pandas_optimized_version(df):
    df_clean = df.dropna(subset=['Sentence']).copy()
    df_clean['sentence_clean'] = df_clean['Sentence'].astype(str).str.strip().str.lower()
    df_grouped = df_clean.groupby('sentence_clean').first().reset_index()
    print(f"✅ Reduced from {len(df_clean)} to {len(df_grouped)} unique sentences")
    return df_grouped

def sqlite_version(df):
    db_fd, db_path = tempfile.mkstemp(suffix='.db')
    conn = sqlite3.connect(db_path)
    df.to_sql('sentences', conn, if_exists='replace', index=False)
    conn.execute('CREATE INDEX IF NOT EXISTS idx_sentence ON sentences(Sentence)')

    batch_size = 1000
    total_rows = conn.execute('SELECT COUNT(*) FROM sentences WHERE Sentence IS NOT NULL').fetchone()[0]

    node_data = defaultdict(set)
    edge_data = defaultdict(set)

    for offset in range(0, total_rows, batch_size):
        cursor = conn.execute("""
            SELECT Sentence, doi, Year, Title, Authors, keywords_from_filename
            FROM sentences
            WHERE Sentence IS NOT NULL
            LIMIT ? OFFSET ?
        """, (batch_size, offset))
        batch = cursor.fetchall()


    conn.close()
    return node_data, edge_data

def cached_nlp_processing(sentences, nlp, cache_file='nlp_cache.pkl', disable=['parser', 'tagger', 'lemmatizer']):
    disable_str = '_'.join(disable)
    hash_key = hashlib.md5((''.join(sentences) + disable_str).encode()).hexdigest()
    cache_path = f"{cache_file}_{hash_key}"

    try:
        with open(cache_path, 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        results = list(nlp.pipe(sentences, disable=disable))
        with open(cache_path, 'wb') as f:
            pickle.dump(results, f)
        return results

def stanza_alternative(sentences):
    import stanza
    nlp = stanza.Pipeline('en', processors='tokenize,ner', use_gpu=True)
    results = []
    for sent in sentences:
        results.append(nlp(sent))
    return results

def ultimate_optimization(df, nlp, use_cache=True, use_parallel=False):
    print("🚀 Starting ultimate optimization...")
    df_opt = pandas_optimized_version(df)
    sentences = df_opt['sentence_clean'].tolist()

    if use_cache:
        nlp_results = cached_nlp_processing(sentences, nlp)
    elif use_parallel:
        nlp_results = parallel_processing_version(df_opt, 'en_ner_bionlp13cg_md')
    else:
        nlp_results = list(nlp.pipe(sentences, disable=['parser', 'tagger', 'lemmatizer']))

    return nlp_results

def build_graph_with_metadata(df):
    node_meta = defaultdict(lambda: {
        'sentences': set(),
        'dois': set(),
        'years': set(),
        'titles': set(),
        'authors': set(),
        'keywords': set()
    })
    edge_meta = defaultdict(lambda: {
        'sentences': set(),
        'dois': set(),
        'years': set(),
        'titles': set(),
        'authors': set(),
        'keywords': set()
    })

    for _, row in df.dropna(subset=['Sentence']).iterrows():
        sent = str(row['Sentence']).strip().lower()
        doi      = row.get('doi')
        year     = row.get('Year')
        title    = row.get('Title')
        authors  = row.get('Authors')
        keywords = row.get('keywords_from_filename')

        doc = nlp(sent)
        ents = list({ent.text.strip().lower() for ent in doc.ents if len(ent.text.strip()) > 2})
        ents = [e for e in ents if e not in nlp.Defaults.stop_words and not e.isdigit()]

        for e in ents:
            node_meta[e]['sentences'].add(sent)
            node_meta[e]['dois'].add(doi)
            node_meta[e]['years'].add(year)
            node_meta[e]['titles'].add(title)
            node_meta[e]['authors'].add(authors)
            node_meta[e]['keywords'].add(keywords)

        for i in range(len(ents)):
            for j in range(i + 1, len(ents)):
                u, v = sorted((ents[i], ents[j]))
                edge_meta[(u, v)]['sentences'].add(sent)
                edge_meta[(u, v)]['dois'].add(doi)
                edge_meta[(u, v)]['years'].add(year)
                edge_meta[(u, v)]['titles'].add(title)
                edge_meta[(u, v)]['authors'].add(authors)
                edge_meta[(u, v)]['keywords'].add(keywords)

    G = nx.Graph()
    for term, meta in node_meta.items():
        G.add_node(term, sentences=list(meta['sentences']))

    for (u, v), meta in edge_meta.items():
        G.add_edge(u, v, weight=len(meta['sentences']), sentences=list(meta['sentences']))

    node_full_meta = {
        term: {k: list(v) for k, v in meta.items() if k != 'sentences'}
        for term, meta in node_meta.items()
    }
    edge_full_meta = {
        (u, v): {k: list(v) for k, v in meta.items() if k != 'sentences'}
        for (u, v), meta in edge_meta.items()
    }

    return G, node_full_meta, edge_full_meta

print("""
""")

In [5]:
G, node_full_meta, edge_full_meta = build_graph_with_metadata(df)


In [13]:
# Теперь:
print("Узлов в графе:", G.number_of_nodes())
print("Рёбер в графе:", G.number_of_edges())

# Пример: получим предложения и метаданные
sample_node = list(G.nodes)[0]
print("Предложения узла", sample_node, ":", G.nodes[sample_node]['sentences'])
print("Метаданные узла", sample_node, ":", node_full_meta[sample_node])


Узлов в графе: 32707
Рёбер в графе: 775715
Предложения узла muscle biopsies : ['increased dna demethylation was also observed during myogenic differentiation of human myoblast obtained from muscle biopsies, which was linked to increased tet1-2 mrna and 5hmc levels [55].', 'at the same time, we were able to evaluate the transcriptomic changes induced by real µg in differentiating humpcs previously isolated from muscle biopsies of the same crew member and an age- and sex-matched volunteer (fig. 1).', 'in a multi-ethnic study of human sarcopenia versus age-matched controls in muscle biopsies, the transcriptome profiles of genes involved in mitochondria biogenesis, e.g., pgc-1α and errα nuclear receptor, were reduced in sarcopenic individuals[161].', 'to complement our previous analyses of muscle biopsies of human sarcopenia3 and understand if mitochondrial dysfunction and altered nad+ metabolism could be linked to systemic changes, we investigated serum levels of the kynurenine / vitamin 

In [67]:
import random
import time
import json
import re
from collections import defaultdict, Counter
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class SpecializedTermSelector:
    def __init__(self, G, node_full_meta):
        """
        Специализированный селектор терминов на основе ваших ключевых слов
        """
        self.G = G
        self.node_full_meta = node_full_meta
        print(f"🔍 Инициализация для графа: {G.number_of_nodes():,} узлов, {G.number_of_edges():,} рёбер")

        # Ваши ключевые слова, разделенные
        self.user_keywords = self._parse_user_keywords()

        # Кэш для ускорения работы
        self._degree_cache = {}
        self._specialized_terms_cache = None
        self._doi_cache = {}

        # Предварительная индексация
        self._build_indices()

        # Создаем специализированные категории
        self.term_categories = self._build_specialized_categories()

    def _parse_user_keywords(self):
        """Парсинг ваших ключевых слов"""
        raw_keywords = [
            '5hmC, human', 'age, related, pathology, human',
            'aging, biomarkers, human', 'aging, biomarkers',
            'aging, genes, human', 'aging, genes', 'aging, human',
            'aging, therapy, human', 'aging, therapy', 'aging, (1)',
            'ATAD3A, human', 'Autophagy, activators', 'autophagy, human',
            'biological, aging, (1)', 'biomarker, of, senescence, human',
            'booster, aging', 'bulk, RNA, human, aging',
            'caloric, restriction, human', 'CD38, human, aging',
            'clonal, expansion, human, aging', 'clonal, expansion, human',
            'COA1', 'COL18A1, human', 'COL18A1',
            'correlates, with, lifespan, human', 'cytokines, human, aging',
            'decreases, with, age, human', 'destruction, of, senescent, cells',
            'DNA, methylation, age', 'DNA, methylation, microbiome',
            'drives, inflammation, human', 'EMILIN1', 'EPHAR',
            'epigenetic, age, human', 'epigenetic, clock, human',
            'epigenetic, clock', 'epigenetic, marks, human', 'frailty, index',
            'GATA4, human', 'GDF15, human',
            'Genes, associated, with, aging, human',
            'Genes, associated, with, aging', 'Gut, aging, human',
            'HMGB1, human', 'HOPX', 'HOPX, human', 'immune, profiling, human',
            'immunosenescence, human', 'increases, with, age, human',
            'inflammaging, human', 'inflammaging, (1)', 'inflammaging, (2)',
            'inflammaging', 'is, a, hallmark, of, aging, human',
            'is, associated, with, aging, human',
            'is, enriched, in, centenarians',
            'is, linked, to, longevity, human', 'KAT7, human, (1)',
            'KAT7, inhibitors', 'longevity, pathways, human',
            'mediates, aging, effects', 'sentences, with, meta',
            'methylation, longevity', 'neuropeptide, human, expression',
            'potential, modulators, human', 'single, cell, Aging, human',
            'T, cell, exhaustion, human', 'TNF, alpha, longevity'
        ]

        # Разделяем по запятым и убираем лишние символы
        all_keywords = set()
        for keyword_group in raw_keywords:
            # Убираем скобки и лишние символы
            clean_group = re.sub(r'[(),]', '', keyword_group)
            # Разделяем по запятым и добавляем отдельные слова
            words = [word.strip().lower() for word in clean_group.split(',') if word.strip()]
            all_keywords.update(words)

        print(f"📋 Извлечено {len(all_keywords)} уникальных ключевых слов")
        return list(all_keywords)

    def _build_indices(self):
        """Предварительная индексация для ускорения"""
        print("🚀 Построение индексов...")

        # Кэш степеней узлов
        self._degree_cache = dict(self.G.degree())

        # Находим специализированные термины
        self._specialized_terms_cache = self._find_specialized_terms()

        # Индекс DOI
        self._build_doi_index()

        print(f"✅ Найдено {len(self._specialized_terms_cache)} специализированных терминов")

    def _find_specialized_terms(self, min_degree=1, max_degree=100):
        """
        Находим термины, соответствующие вашим ключевым словам
        """
        print("🔍 Поиск специализированных терминов по вашим ключевым словам...")

        specialized_terms = []

        for node in self.G.nodes():
            degree = self._degree_cache.get(node, 0)

            # Фильтр по степени
            if not (min_degree <= degree <= max_degree):
                continue

            # Проверяем соответствие вашим ключевым словам
            if not self._matches_user_keywords(node):
                continue

            # Базовые фильтры качества
            if not self._is_quality_term(node):
                continue

            # Вычисляем специализированный скор
            spec_score = self._calculate_specialized_score(node)

            specialized_terms.append((node, degree, spec_score))

        # Сортируем по специализированному скору
        specialized_terms.sort(key=lambda x: x[2], reverse=True)
        return [term for term, _, _ in specialized_terms[:1500]]  # Топ-1500

    def _matches_user_keywords(self, term):
        """Проверяем соответствие термина вашим ключевым словам"""
        term_lower = term.lower()

        # Прямое соответствие любому ключевому слову
        for keyword in self.user_keywords:
            if keyword in term_lower:
                return True

        # Дополнительные паттерны на основе ваших ключевых слов
        aging_patterns = [
            r'aging', r'senescence', r'longevity', r'lifespan',
            r'biomarker', r'epigenetic', r'inflammaging', r'immunosenescence',
            r'methylation', r'autophagy', r'frailty', r'centenarian'
        ]

        for pattern in aging_patterns:
            if re.search(pattern, term_lower):
                return True

        return False

    def _is_quality_term(self, term):
        """Проверка базового качества термина"""
        if len(term) < 3 or len(term) > 60:
            return False

        term_lower = term.lower()

        # Исключаем технические термины
        exclude_patterns = [
            r'^\d+$', r'figure\s*\d+', r'table\s*\d+',
            r'\.jpg$', r'\.png$', r'\.pdf$',
            r'^[a-z]\d+$', r'supplementary', r'additional',
            r'method', r'protocol', r'dataset'
        ]

        for pattern in exclude_patterns:
            if re.search(pattern, term_lower):
                return False

        return True

    def _calculate_specialized_score(self, node):
        """
        Специализированный скор на основе ваших критериев
        """
        score = 0
        term_lower = node.lower()

        # Бонусы за ключевые термины вашего исследования
        high_value_keywords = {
            'human': 10, 'aging': 8, 'biomarker': 7, 'epigenetic': 6,
            'senescence': 6, 'longevity': 8, 'inflammaging': 7,
            'immunosenescence': 6, 'methylation': 5, 'autophagy': 5,
            'frailty': 5, 'centenarian': 8, 'clock': 6, 'therapy': 5
        }

        for keyword, bonus in high_value_keywords.items():
            if keyword in term_lower:
                score += bonus

        # Бонус за специфичные гены/белки из ваших ключевых слов
        specific_genes = [
            'atad3a', 'col18a1', 'coa1', 'emilin1', 'ephar',
            'gata4', 'gdf15', 'hmgb1', 'hopx', 'kat7', 'cd38'
        ]

        for gene in specific_genes:
            if gene in term_lower:
                score += 15  # Высокий бонус за специфичные гены

        # Бонус за редкость (обратная степень)
        degree = self._degree_cache.get(node, 0)
        rarity_bonus = max(0, 25 - degree)
        score += rarity_bonus

        # Бонус за наличие DOI
        if node in self.node_full_meta:
            meta = self.node_full_meta[node]
            if isinstance(meta, dict) and meta.get('doi'):
                score += 8

        # Бонус за разнообразие соседей
        neighbors = list(self.G.neighbors(node))
        if neighbors:
            neighbor_degrees = [self._degree_cache.get(n, 0) for n in neighbors]
            diversity = len(set(neighbor_degrees)) / len(neighbor_degrees)
            score += diversity * 5

        return score

    def _build_doi_index(self):
        """Построение индекса DOI с учетом вашей структуры метаданных"""
        print("📚 Построение индекса DOI...")

        # Отладка: смотрим на примеры метаданных
        sample_nodes = list(self.node_full_meta.keys())[:3]
        print("🔍 Примеры метаданных для отладки:")
        for node in sample_nodes:
            meta = self.node_full_meta.get(node)
            print(f"   {node}: {type(meta)}")
            if isinstance(meta, dict):
                print(f"      ключи: {list(meta.keys())}")
                if 'dois' in meta:
                    print(f"      dois: {meta['dois'][:3] if len(meta['dois']) > 3 else meta['dois']}")

        # Поля для поиска DOI в вашей структуре
        doi_fields_to_check = ['dois', 'doi', 'DOI', 'Doi', 'pmid', 'PMID', 'pubmed_id', 'publication_id']

        for node, meta in self.node_full_meta.items():
            if isinstance(meta, dict):
                # Проверяем разные поля с DOI
                for field in doi_fields_to_check:
                    if field in meta:
                        field_value = meta[field]

                        # Если это список (как в вашей структуре)
                        if isinstance(field_value, list):
                            # Берем первый непустой DOI из списка
                            for doi_item in field_value:
                                if doi_item and str(doi_item).strip() and str(doi_item).strip().lower() not in ['none', 'nan', 'null']:
                                    self._doi_cache[node] = str(doi_item).strip()
                                    break
                            if node in self._doi_cache:
                                break

                        # Если это строка
                        elif isinstance(field_value, str) and field_value.strip() and field_value.strip().lower() not in ['none', 'nan', 'null']:
                            self._doi_cache[node] = field_value.strip()
                            break

            elif isinstance(meta, str) and ('doi' in meta.lower() or 'pmid' in meta.lower()):
                # Если метаданные - строка с DOI
                self._doi_cache[node] = meta[:50]  # Берем первые 50 символов

        print(f"✅ Найдено {len(self._doi_cache)} узлов с DOI/PMID")
        if len(self._doi_cache) > 0:
            print("🔍 Примеры найденных DOI:")
            for i, (node, doi) in enumerate(list(self._doi_cache.items())[:5]):
                print(f"   {node}: {doi}")
        else:
            print("⚠️ DOI не найдены. Структура метаданных:")
            print("   Ожидается: node_full_meta[term]['dois'] как список")
            print("   Или: node_full_meta[term]['doi'] как строка")

    def _count_specialized_neighbors(self, term, min_count=3):
        """
        Подсчет специализированных соседей термина
        Возвращает количество соседей, соответствующих нашим ключевым словам
        """
        if term not in self.G:
            return 0

        neighbors = list(self.G.neighbors(term))
        specialized_count = 0

        for neighbor in neighbors:
            if self._matches_user_keywords(neighbor):
                specialized_count += 1

        return specialized_count

    def _build_specialized_categories(self, min_specialized_neighbors=1):
        """Построение специализированных категорий с расширенными ключевыми словами"""
        print("🔧 Построение специализированных категорий с расширенными ключевыми словами...")
        print(f"🎯 Минимальный порог специализированных соседей: {min_specialized_neighbors}")

        categories = {
            '🧬 Биомаркеры старения человека': {
                'keywords': [
                    'biomarker', 'aging', 'human', 'senescence', 'frailty', 'age',
                    'elderly', 'old', 'lifespan', 'longevity', 'mortality', 'centenarian',
                    'gerontology', 'geriatric', 'adult', 'cohort'
                ],
                'terms': []
            },
            '🔬 Эпигенетические часы': {
                'keywords': [
                    'epigenetic', 'clock', 'methylation', 'dna', 'histone', 'chromatin',
                    'cpg', 'dnmt', 'tet', 'demethylation', 'hypermethylation',
                    'h3k4', 'h3k27', 'h3k9', 'acetylation', 'modification'
                ],
                'terms': []
            },
            '⚡ Воспаление при старении': {
                'keywords': [
                    'inflammaging', 'inflammation', 'inflammatory', 'immune', 'cytokine',
                    'interleukin', 'tnf', 'il-1', 'il-6', 'il-8', 'nfkb', 'nf-kb',
                    'macrophage', 'neutrophil', 'monocyte', 'innate', 'adaptive',
                    'interferon', 'chemokine'
                ],
                'terms': []
            },
            '🧪 Гены и белки старения': {
                'keywords': [
                    'gene', 'protein', 'enzyme', 'kinase', 'phosphatase', 'receptor',
                    'transcription', 'translation', 'expression', 'regulation',
                    'p53', 'p21', 'rb', 'foxo', 'sirt', 'mtor', 'ampk', 'insulin',
                    'growth', 'factor', 'hormone', 'pathway', 'signaling'
                ],
                'terms': []
            },
            '🏥 Терапевтические мишени': {
                'keywords': [
                    'therapy', 'treatment', 'drug', 'compound', 'inhibitor', 'activator',
                    'modulator', 'intervention', 'pharmaceutical', 'medicine',
                    'therapeutic', 'clinical', 'trial', 'dosage', 'administration'
                ],
                'terms': []
            },
            '🧠 Иммунное старение': {
                'keywords': [
                    'immunosenescence', 'immune', 'immunity', 'lymphocyte', 'tcell', 'bcell',
                    't cell', 'b cell', 'cd4', 'cd8', 'cd28', 'cd57', 'exhaustion',
                    'memory', 'naive', 'effector', 'regulatory', 'treg', 'th1', 'th2'
                ],
                'terms': []
            },
            '🔀 Пути долголетия': {
                'keywords': [
                    'longevity', 'lifespan', 'survival', 'mortality', 'centenarian',
                    'pathway', 'signaling', 'cascade', 'network', 'regulation',
                    'homeostasis', 'maintenance', 'repair', 'protection'
                ],
                'terms': []
            },
            '📊 Клеточные процессы': {
                'keywords': [
                    'cell', 'cellular', 'mitochondria', 'autophagy', 'apoptosis',
                    'proliferation', 'differentiation', 'stem', 'division', 'cycle',
                    'metabolism', 'respiration', 'oxidative', 'stress', 'damage',
                    'repair', 'maintenance'
                ],
                'terms': []
            }
        }

        # ДИАГНОСТИКА: анализируем проблему с расширенными ключевыми словами
        print(f"\n🔍 ДИАГНОСТИКА С РАСШИРЕННЫМИ КЛЮЧЕВЫМИ СЛОВАМИ:")

        # Функция для более гибкого поиска
        def flexible_match(term, keywords):
            term_lower = term.lower()
            # Прямое совпадение
            for keyword in keywords:
                if keyword in term_lower:
                    return True

            # Дополнительные паттерны
            bio_patterns = [
                r'\baging\b', r'\bhuman\b', r'\bcell\b', r'\bgene\b', r'\bprotein\b',
                r'\binflammation\b', r'\bimmune\b', r'\bmethylation\b', r'\bepigenetic\b',
                r'\blongevity\b', r'\bsenescence\b', r'\btherapy\b', r'\btreatment\b'
            ]

            for pattern in bio_patterns:
                if re.search(pattern, term_lower):
                    return True

            return False

        # Анализируем с новыми критериями
        neighbor_stats = []
        for i, term in enumerate(self._specialized_terms_cache[:20]):
            total_neighbors = len(list(self.G.neighbors(term))) if term in self.G else 0
            # Используем гибкий поиск
            all_keywords = []
            for cat_keywords in categories.values():
                all_keywords.extend(cat_keywords['keywords'])

            matches_flexible = flexible_match(term, all_keywords)
            specialized_neighbors = self._count_specialized_neighbors(term) if matches_flexible else 0

            neighbor_stats.append((term, total_neighbors, specialized_neighbors, matches_flexible))
            status = "✅" if matches_flexible else "❌"
            print(f"   {i+1:2}. {term:<35} {status} (всего: {total_neighbors}, спец: {specialized_neighbors})")

        # Статистика по всем терминам с гибким поиском
        flexible_matching_terms = []
        for term in self._specialized_terms_cache:
            all_keywords = []
            for cat_keywords in categories.values():
                all_keywords.extend(cat_keywords['keywords'])

            if flexible_match(term, all_keywords):
                flexible_matching_terms.append(term)

        print(f"\n📊 СТАТИСТИКА С РАСШИРЕННЫМИ КРИТЕРИЯМИ:")
        print(f"   Терминов соответствует расширенным критериям: {len(flexible_matching_terms)}")

        # Если все еще мало, добавляем самые популярные термины
        if len(flexible_matching_terms) < 50:
            print(f"⚠️ Все еще мало подходящих терминов. Добавляем популярные биологические термины...")

            # Добавляем термины с высокой степенью связности
            high_degree_terms = []
            for term in self.G.nodes():
                degree = self._degree_cache.get(term, 0)
                if degree >= 10:  # Популярные термины
                    # Простая проверка на биологичность
                    if any(bio_word in term.lower() for bio_word in [
                        'cell', 'gene', 'protein', 'human', 'aging', 'age', 'old',
                        'inflammation', 'immune', 'therapy', 'treatment', 'disease',
                        'cancer', 'diabetes', 'expression', 'regulation', 'pathway',
                        'signaling', 'metabolism', 'mitochondria', 'dna', 'rna'
                    ]):
                        high_degree_terms.append((term, degree))

            # Сортируем по популярности
            high_degree_terms.sort(key=lambda x: x[1], reverse=True)
            popular_terms = [term for term, _ in high_degree_terms[:100]]

            print(f"✅ Добавлено {len(popular_terms)} популярных биологических терминов")
            flexible_matching_terms.extend(popular_terms)

        # Счетчики для статистики
        filtered_out_count = 0
        total_candidates = len(flexible_matching_terms)

        # Распределяем термины по категориям с гибким поиском
        for term in flexible_matching_terms:
            term_lower = term.lower()

            # АДАПТИВНЫЙ ФИЛЬТР
            if min_specialized_neighbors > 0:
                all_keywords = []
                for cat_keywords in categories.values():
                    all_keywords.extend(cat_keywords['keywords'])

                if flexible_match(term, all_keywords):
                    specialized_neighbors_count = self._count_specialized_neighbors(term)
                else:
                    specialized_neighbors_count = 0

                if specialized_neighbors_count < min_specialized_neighbors:
                    filtered_out_count += 1
                    continue
            else:
                # Если специализированных соседей нет, проверяем общие соседи
                total_neighbors = len(list(self.G.neighbors(term))) if term in self.G else 0
                if total_neighbors < 5:  # Минимум 5 общих соседей для популярных терминов
                    filtered_out_count += 1
                    continue

            # Распределяем по категориям с гибким поиском
            added_to_category = False
            for category, data in categories.items():
                if not added_to_category and flexible_match(term, data['keywords']):
                    data['terms'].append(term)
                    added_to_category = True

        print(f"\n🚫 Отфильтровано {filtered_out_count} из {total_candidates} терминов")

        # Ограничиваем и сортируем по специализированному скору
        for category, data in categories.items():
            if len(data['terms']) > 15:  # Уменьшаем лимит для лучшего качества
                terms_with_score = []
                for term in data['terms']:
                    score = self._calculate_specialized_score(term)
                    degree = self._degree_cache.get(term, 0)
                    # Комбинированный скор: специализация + популярность
                    combined_score = score + (degree * 0.5)
                    terms_with_score.append((term, combined_score))

                terms_with_score.sort(key=lambda x: x[1], reverse=True)
                data['terms'] = [term for term, _ in terms_with_score[:15]]

        # Финальная статистика
        total_quality_terms = sum(len(data['terms']) for data in categories.values())
        print(f"✅ В категории добавлено {total_quality_terms} качественных терминов")

        # Показываем статистику по категориям
        print(f"\n📊 СТАТИСТИКА ПО КАТЕГОРИЯМ (с расширенными ключевыми словами):")
        for category, data in categories.items():
            terms_count = len(data['terms'])
            if terms_count > 0:
                # Показываем средние характеристики
                avg_degree = sum(self._degree_cache.get(term, 0) for term in data['terms'][:3]) / min(3, terms_count)
                print(f"   {category}: {terms_count} терминов (средняя степень: {avg_degree:.1f})")
            else:
                print(f"   {category}: 0 терминов ⚠️")

        return categories

    def get_term_metadata(self, term):
        """Получить расширенные метаданные термина с учетом вашей структуры"""
        metadata = {
            'term': term,
            'degree': self._degree_cache.get(term, 0),
            'doi': self._doi_cache.get(term),
            'neighbors_count': len(list(self.G.neighbors(term))) if term in self.G else 0,
            'specialized_score': self._calculate_specialized_score(term),
            'matches_keywords': []
        }

        # Проверяем какие ключевые слова совпадают
        term_lower = term.lower()
        for keyword in self.user_keywords:
            if keyword in term_lower:
                metadata['matches_keywords'].append(keyword)

        # Дополнительные метаданные из node_full_meta с учетом вашей структуры
        if term in self.node_full_meta:
            node_meta = self.node_full_meta[term]
            if isinstance(node_meta, dict):
                # Извлекаем первые элементы из списков (если они есть)
                titles = node_meta.get('titles', [])
                authors = node_meta.get('authors', [])
                years = node_meta.get('years', [])
                keywords = node_meta.get('keywords', [])
                dois = node_meta.get('dois', [])

                metadata.update({
                    'title': titles[0] if titles and titles[0] else None,
                    'authors': authors[0] if authors and authors[0] else None,
                    'year': years[0] if years and years[0] else None,
                    'source_keywords': keywords[0] if keywords and keywords[0] else None,
                    'doi_from_meta': dois[0] if dois and dois[0] else None,
                    'total_papers': len([d for d in dois if d and str(d).strip()]) if dois else 0,
                    'year_range': f"{min(years) if years else 'N/A'}-{max(years) if years else 'N/A'}",
                    'type': 'biomedical_entity'
                })

                # Формируем описание
                if titles and titles[0]:
                    title_text = str(titles[0])[:150]
                    metadata['description'] = f"Biomedical entity from: {title_text}..."
                else:
                    metadata['description'] = f"Biomedical entity with {len(dois) if dois else 0} references"

        return metadata

    def show_categories_specialized(self):
        """Показать специализированные категории"""
        print("\n🎯 СПЕЦИАЛИЗИРОВАННЫЕ КАТЕГОРИИ ПО ВАШИМ КЛЮЧЕВЫМ СЛОВАМ")
        print("=" * 80)

        for i, (category, data) in enumerate(self.term_categories.items(), 1):
            terms = data['terms']
            print(f"\n{i}. {category} ({len(terms)} терминов)")

            if terms:
                print("   Топ-3 по специализированному скору:")
                for j, term in enumerate(terms[:3], 1):
                    degree = self._degree_cache.get(term, 0)
                    doi_status = "📚" if term in self._doi_cache else "❌"
                    score = self._calculate_specialized_score(term)
                    print(f"   {j}. {term} (связей: {degree}, DOI: {doi_status}, скор: {score:.1f})")
            else:
                print("   Термины не найдены")

        print(f"\n0. ✍️  Ввести термин вручную")
        print(f"99. 🎲 Случайный специализированный термин")

    def show_term_details(self, term):
        """Подробная информация о термине с вашими ключевыми словами"""
        print(f"\n📋 ДЕТАЛЬНАЯ ИНФОРМАЦИЯ: {term}")
        print("=" * 60)

        metadata = self.get_term_metadata(term)

        print(f"🔗 Степень узла: {metadata['degree']}")
        print(f"🎯 Специализированный скор: {metadata['specialized_score']:.1f}")
        print(f"👥 Соседей: {metadata['neighbors_count']}")

        if metadata['doi']:
            print(f"📚 DOI: {metadata['doi']}")
        else:
            print(f"📚 DOI: не найден")

        if metadata['matches_keywords']:
            print(f"🔑 Совпадения с вашими ключевыми словами: {', '.join(metadata['matches_keywords'][:5])}")

        # Дополнительная информация из вашей структуры
        if metadata.get('title'):
            print(f"📄 Заголовок: {metadata['title'][:100]}...")

        if metadata.get('authors'):
            print(f"👨‍🔬 Авторы: {metadata['authors'][:100]}...")

        if metadata.get('year'):
            print(f"📅 Год: {metadata['year']}")

        if metadata.get('year_range'):
            print(f"📊 Диапазон лет: {metadata['year_range']}")

        if metadata.get('total_papers'):
            print(f"📚 Всего статей: {metadata['total_papers']}")

        if metadata.get('source_keywords'):
            print(f"🏷️  Ключевые слова источника: {metadata['source_keywords'][:100]}...")

        if metadata.get('description'):
            print(f"📝 Описание: {metadata['description']}")

        # Показываем специализированных соседей
        if term in self.G:
            neighbors = list(self.G.neighbors(term))
            if neighbors:
                # Сортируем соседей по специализированному скору
                neighbor_scores = [(n, self._calculate_specialized_score(n)) for n in neighbors[:15]]
                neighbor_scores.sort(key=lambda x: x[1], reverse=True)

                print(f"\n🔗 Топ-5 специализированных соседей:")
                for i, (neighbor, score) in enumerate(neighbor_scores[:5], 1):
                    doi_status = "📚" if neighbor in self._doi_cache else "❌"
                    degree = self._degree_cache.get(neighbor, 0)
                    print(f"   {i}. {neighbor} (скор: {score:.1f}, связей: {degree}, DOI: {doi_status})")

    def get_category_terms(self, category_num):
        """Получить термины из категории"""
        categories_list = list(self.term_categories.items())

        if 1 <= category_num <= len(categories_list):
            category_name, data = categories_list[category_num - 1]
            return category_name, data['terms']

        return None, []

    def get_random_term(self):
        """Получить случайный качественный термин (≥3 специализированных соседей)"""
        quality_terms = []

        # Собираем все термины из категорий (они уже прошли фильтрацию)
        for data in self.term_categories.values():
            quality_terms.extend(data['terms'])

        if quality_terms:
            return random.choice(quality_terms)
        else:
            # Если в категориях пусто, ищем среди всех специализированных терминов
            print("⚠️ Поиск качественных терминов среди всех...")
            for term in self._specialized_terms_cache:
                if self._count_specialized_neighbors(term) >= 3:
                    quality_terms.append(term)

            if quality_terms:
                print(f"✅ Найдено {len(quality_terms)} качественных терминов")
                return random.choice(quality_terms)
            else:
                print("❌ Качественные термины не найдены")
                return None

    def validate_manual_term(self, term):
        """
        Валидация ручного ввода термина
        Возвращает (is_valid, warning_message, quality_info)
        """
        if term not in self.G:
            return False, f"Термин '{term}' не найден в графе", {}

        # Подсчитываем качественные показатели
        degree = self._degree_cache.get(term, 0)
        specialized_neighbors = self._count_specialized_neighbors(term)
        matches_keywords = self._matches_user_keywords(term)
        has_doi = term in self._doi_cache
        spec_score = self._calculate_specialized_score(term)

        quality_info = {
            'degree': degree,
            'specialized_neighbors': specialized_neighbors,
            'matches_keywords': matches_keywords,
            'has_doi': has_doi,
            'spec_score': spec_score
        }

        # Проверяем критерии качества
        warnings = []
        is_high_quality = True

        if specialized_neighbors < 3:
            warnings.append(f"❌ Мало специализированных соседей: {specialized_neighbors} < 3")
            is_high_quality = False

        if not matches_keywords:
            warnings.append(f"⚠️ Не соответствует вашим ключевым словам")

        if not has_doi:
            warnings.append(f"⚠️ Отсутствует DOI")

        if degree < 2:
            warnings.append(f"⚠️ Очень мало связей: {degree}")

        if spec_score < 10:
            warnings.append(f"⚠️ Низкий специализированный скор: {spec_score:.1f}")

        # Формируем сообщение
        if is_high_quality and not warnings:
            message = f"✅ Качественный термин готов для анализа"
        elif is_high_quality:
            message = f"⚠️ Термин пригоден, но есть замечания:\n" + "\n".join([f"   {w}" for w in warnings])
        else:
            message = f"❌ Термин не рекомендуется для качественного анализа:\n" + "\n".join([f"   {w}" for w in warnings])

        return is_high_quality, message, quality_info

class EnhancedHypothesisGenerator:
    def __init__(self, G, node_full_meta, edge_full_meta, client):
        self.G = G
        self.node_full_meta = node_full_meta
        self.edge_full_meta = edge_full_meta
        self.client = client

        # Мягкие критерии качества (не блокирующие)
        self.quality_metrics = {
            'preferred_neighbors': 3,      # Предпочтительно, но не обязательно
            'preferred_doi_count': 1,      # Хорошо иметь, но не критично
            'preferred_spec_score': 10     # Рекомендуемый, но гибкий
        }

    def generate_hypotheses(self, term, count=5, temperature=0.7):
        """Генерация гипотез БЕЗ строгой фильтрации"""
        print(f"\n💡 ГЕНЕРАЦИЯ {count} ГИПОТЕЗ ДЛЯ '{term}'")
        print(f"🌡️ Температура: {temperature}")
        print("-" * 60)

        # 1. Мягкая оценка качества (информативная, не блокирующая)
        quality_info = self.assess_term_quality(term)
        self._display_quality_info(quality_info)

        # 2. Анализ связей (берем что есть)
        connections = self.analyze_connections(term)

        if not connections['neighbors']:
            print("⚠️ У термина нет соседей. Генерируем гипотезы на основе метаданных...")

        # 3. Генерация гипотез (всегда пытаемся сгенерировать)
        hypotheses = []
        attempts = 0
        max_attempts = count * 3  # Больше попыток

        while len(hypotheses) < count and attempts < max_attempts:
            attempts += 1
            hypothesis = self._generate_single_hypothesis(term, connections, temperature)

            # Мягкая проверка (исправляем, а не отбрасываем)
            if hypothesis:
                validated_hypothesis = self._enhance_hypothesis(hypothesis, term, connections)
                hypotheses.append(validated_hypothesis)

        # 4. Ранжирование по качеству
        ranked = self._rank_hypotheses(hypotheses)

        print(f"✅ Сгенерировано {len(ranked)} гипотез")

        return {
            'term': term,
            'quality_info': quality_info,
            'connections': connections,
            'hypotheses': ranked[:count],
            'generation_temperature': temperature,
            'total_attempts': attempts,
            'success_rate': len(ranked) / attempts if attempts > 0 else 0
        }

    def assess_term_quality(self, term):
        """Мягкая оценка качества (информативная, не блокирующая)"""
        if term not in self.G:
            return {
                'exists': False,
                'message': "Термин не найден в графе",
                'can_generate': False
            }

        degree = self.G.degree(term)
        doi_count = self._count_doi(term)
        spec_score = self._calculate_specialized_score(term)

        # Мягкие рекомендации (не требования)
        quality_level = "отличное"
        recommendations = []

        if degree < self.quality_metrics['preferred_neighbors']:
            quality_level = "хорошее" if degree >= 2 else "ограниченное"
            recommendations.append(f"Больше связей улучшит контекст ({degree} есть)")

        if doi_count < self.quality_metrics['preferred_doi_count']:
            recommendations.append(f"DOI источники укрепят обоснование ({doi_count} найдено)")

        if spec_score < self.quality_metrics['preferred_spec_score']:
            recommendations.append(f"Специализированный скор можно улучшить ({spec_score:.1f})")

        return {
            'exists': True,
            'can_generate': True,  # Всегда можем попробовать
            'quality_level': quality_level,
            'message': f"Качество данных: {quality_level}",
            'recommendations': recommendations,
            'metrics': {
                'degree': degree,
                'doi_count': doi_count,
                'spec_score': spec_score
            }
        }

    def _display_quality_info(self, quality_info):
        """Показ информации о качестве"""
        if not quality_info['exists']:
            print(f"❌ {quality_info['message']}")
            return

        print(f"📊 {quality_info['message']}")

        metrics = quality_info['metrics']
        print(f"   🔗 Связей: {metrics['degree']}")
        print(f"   📚 DOI источников: {metrics['doi_count']}")
        print(f"   🎯 Специализированный скор: {metrics['spec_score']:.1f}")

        if quality_info['recommendations']:
            print(f"💡 Рекомендации для улучшения:")
            for rec in quality_info['recommendations']:
                print(f"   • {rec}")

    def analyze_connections(self, term):
        """Анализ связей без фильтрации"""
        if term not in self.G:
            return {'neighbors': [], 'total_neighbors': 0}

        all_neighbors = list(self.G.neighbors(term))

        # Берем всех соседей, оцениваем их
        neighbor_info = []
        for neighbor in all_neighbors:
            degree = self.G.degree(neighbor)
            doi_count = self._count_doi(neighbor)
            spec_score = self._calculate_specialized_score(neighbor)

            neighbor_info.append({
                'term': neighbor,
                'degree': degree,
                'doi_count': doi_count,
                'spec_score': spec_score,
                'has_doi': doi_count > 0
            })

        # Сортируем по комбинированному скору (степень + DOI + специализация)
        neighbor_info.sort(key=lambda x: x['degree'] + x['doi_count']*5 + x['spec_score'], reverse=True)

        return {
            'neighbors': neighbor_info,
            'total_neighbors': len(all_neighbors),
            'high_quality_neighbors': len([n for n in neighbor_info if n['has_doi']]),
            'avg_degree': sum(n['degree'] for n in neighbor_info) / len(neighbor_info) if neighbor_info else 0
        }

    def _generate_single_hypothesis(self, term, connections, temperature):
        """Генерация одной гипотезы с гибким подходом"""
        # Выбираем лучших соседей (или всех, если мало)
        neighbors = connections['neighbors'][:8]  # Топ-8 или все что есть

        # Получаем DOI информацию
        doi_sources = []
        for neighbor in neighbors:
            if neighbor['has_doi']:
                neighbor_doi = self._get_doi(neighbor['term'])
                if neighbor_doi:
                    doi_sources.append({'term': neighbor['term'], 'doi': neighbor_doi})

        # Создаем адаптивный промпт
        prompt = self._create_adaptive_prompt(term, neighbors, doi_sources, len(connections['neighbors']))

        try:
            response = self.client.chat.completions.create(
                model="llama3.1",
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature,
                max_tokens=4000
            )

            response_text = response.choices[0].message.content.strip()

            # Парсинг JSON с исправлениями
            hypothesis = self._parse_hypothesis_response(response_text)

            if hypothesis:
                hypothesis['source_terms'] = [term] + [n['term'] for n in neighbors[:3]]
                hypothesis['doi_sources'] = doi_sources
                return hypothesis

        except Exception as e:
            print(f"⚠️ Ошибка при генерации: {e}")

        # Резервная гипотеза если основная не удалась
        return self._create_fallback_hypothesis(term, neighbors)

    def _create_adaptive_prompt(self, term, neighbors, doi_sources, total_neighbors):
        """Создание адаптивного промпта в зависимости от доступных данных"""

        neighbor_list = ", ".join([n['term'] for n in neighbors[:5]])

        # Адаптируем промпт к качеству данных
        if len(neighbors) >= 5 and doi_sources:
            # Полный промпт для богатых данных
            context_note = f"У термина {total_neighbors} связей, много контекста"
            doi_note = f"\nДОСТУПНЫЕ DOI ИСТОЧНИКИ:\n" + "\n".join([f"- {d['term']}: {d['doi']}" for d in doi_sources[:3]])
        elif len(neighbors) >= 3:
            # Средний промпт
            context_note = f"У термина {total_neighbors} связей, умеренный контекст"
            doi_note = f"\nDOI источники: {len(doi_sources)} доступно" if doi_sources else "\nDOI ограничены"
        else:
            # Минимальный промпт
            context_note = f"У термина {total_neighbors} связей, ограниченный контекст"
            doi_note = "\nРаботаем с ограниченными данными"

        return f"""Ты — эксперт по биологии старения. Создай КОНКРЕТНУЮ гипотезу.

ОСНОВНОЙ ТЕРМИН: "{term}"
СВЯЗАННЫЕ ТЕРМИНЫ: {neighbor_list}
КОНТЕКСТ: {context_note}{doi_note}

ТРЕБОВАНИЯ - МАКСИМАЛЬНАЯ КОНКРЕТИКА:
🧬 МОЛЕКУЛЯРНЫЕ ДЕТАЛИ:
- Конкретные белки, гены, пути
- Точные концентрации, активности
- Специфичные взаимодействия

🔬 ИЗМЕРИМЫЕ ПАРАМЕТРЫ:
- Конкретные биомаркеры с единицами (нг/мл, мкМ, %)
- Временные рамки изменений
- Пороговые значения

🧪 ЭКСПЕРИМЕНТАЛЬНАЯ ПРОВЕРКА:
- Конкретные методы (qPCR, Western blot, ELISA)
- Детали протокола
- Ожидаемые результаты

ФОРМАТ JSON:
{{
    "hypothesis": "Конкретная гипотеза с точными молекулярными механизмами",
    "mechanism": "Детальный механизм с белками, концентрациями, временными рамками",
    "measurements": ["биомаркер1 (единицы измерения)", "биомаркер2 (единицы)", "функциональный тест"],
    "experimental_design": "Подробный протокол: модель, методы, временные точки, контроли",
    "validation": "Как именно проверить: какие методы, какие значения ожидать",
    "terms_used": ["{term}", "связанный_термин1", "связанный_термин2"],
    "biological_plausibility": "Обоснование с известными фактами",
    "clinical_relevance": "Связь с клиническими проявлениями старения",
    "novelty_score": 8
}}

Генерируй ТОЛЬКО JSON без комментариев!"""

    def _parse_hypothesis_response(self, response_text):
        """Парсинг ответа модели с исправлениями"""
        try:
            return json.loads(response_text)
        except json.JSONDecodeError:
            # Пытаемся извлечь JSON из текста
            start_idx = response_text.find('{')
            end_idx = response_text.rfind('}') + 1
            if start_idx != -1 and end_idx != -1:
                json_text = response_text[start_idx:end_idx]
                try:
                    return json.loads(json_text)
                except json.JSONDecodeError:
                    pass
            return None

    def _enhance_hypothesis(self, hypothesis, term, connections):
        """Улучшение гипотезы (исправление недостатков вместо отбрасывания)"""
        enhanced = dict(hypothesis)

        # Исправляем отсутствующие поля
        if 'measurements' not in enhanced or not enhanced['measurements']:
            enhanced['measurements'] = self._generate_default_measurements(term)

        if 'terms_used' not in enhanced or not enhanced['terms_used']:
            enhanced['terms_used'] = [term] + [n['term'] for n in connections['neighbors'][:2]]

        if 'biological_plausibility' not in enhanced:
            enhanced['biological_plausibility'] = f"Основано на связях {term} в биологической сети"

        # Добавляем метаданные о качестве
        enhanced['quality_metrics'] = {
            'has_measurements': bool(enhanced.get('measurements')),
            'uses_network_terms': any(t in [n['term'] for n in connections['neighbors']] for t in enhanced.get('terms_used', [])),
            'has_doi_support': bool(enhanced.get('doi_sources'))
        }

        return enhanced

    def _generate_default_measurements(self, term):
        """Генерация базовых измерений если они отсутствуют"""
        return [
            f"{term} expression level (fold change)",
            f"{term} protein concentration (ng/ml)",
            f"{term} activity assay (AU/min)"
        ]

    def _create_fallback_hypothesis(self, term, neighbors):
        """Создание резервной гипотезы"""
        neighbor_terms = [n['term'] for n in neighbors[:3]] if neighbors else ["related factors"]

        return {
            "hypothesis": f"{term} plays a role in aging through interactions with {', '.join(neighbor_terms)}",
            "mechanism": f"The {term} pathway interacts with cellular aging processes via network connections",
            "measurements": self._generate_default_measurements(term),
            "experimental_design": f"Study {term} expression and activity in aging models, compare with controls",
            "validation": f"Measure {term} levels in young vs old samples, functional assays",
            "terms_used": [term] + neighbor_terms,
            "biological_plausibility": f"Network analysis suggests {term} is connected to aging-related processes",
            "clinical_relevance": f"Changes in {term} may serve as biomarkers of aging",
            "novelty_score": 6,
            "is_fallback": True
        }

    def _rank_hypotheses(self, hypotheses):
        """Ранжирование гипотез по качеству"""
        def score_hypothesis(h):
            score = 0

            # Бонус за конкретные измерения
            measurements = h.get('measurements', [])
            score += sum(2 for m in measurements if '(' in m and ')' in m)  # Есть единицы

            # Бонус за использование DOI-терминов
            doi_sources = h.get('doi_sources', [])
            score += len(doi_sources) * 3

            # Бонус за детальность механизма
            mechanism = h.get('mechanism', '')
            score += min(len(mechanism) // 100, 5)  # До 5 баллов за детальность

            # Бонус за экспериментальную проверяемость
            validation = h.get('validation', '')
            score += min(len(validation) // 50, 3)

            # Штраф за резервную гипотезу
            if h.get('is_fallback'):
                score -= 5

            # Бонус за новизну
            novelty = h.get('novelty_score', 5)
            score += novelty

            return score

        return sorted(hypotheses, key=score_hypothesis, reverse=True)

    def display_hypotheses(self, result):
        """Показ гипотез с расширенной информацией"""
        if not result or 'hypotheses' not in result:
            print("❌ Нет гипотез для отображения")
            return

        hypotheses = result['hypotheses']
        term = result['term']

        print(f"\n📚 ГИПОТЕЗЫ ДЛЯ '{term.upper()}'")
        print(f"🌡️ Температура: {result.get('generation_temperature', 'неизвестно')}")
        print(f"📊 Успешность: {result.get('success_rate', 0):.1%} ({len(hypotheses)} из {result.get('total_attempts', 0)} попыток)")
        print("=" * 80)

        for i, hyp in enumerate(hypotheses, 1):
            print(f"\n{i}. ГИПОТЕЗА:")
            print(f"   🧬 Термины: {', '.join(hyp.get('terms_used', []))}")
            print(f"   🔬 Механизм: {hyp.get('hypothesis', '')}")
            print(f"   📚 Детали: {hyp.get('mechanism', '')}")

            # Измерения
            measurements = hyp.get('measurements', [])
            if measurements:
                print(f"   📊 Измерения:")
                for j, measurement in enumerate(measurements, 1):
                    print(f"      {j}. {measurement}")

            # Экспериментальная проверка
            if 'experimental_design' in hyp:
                print(f"   🧪 Эксперимент: {hyp['experimental_design']}")

            if 'validation' in hyp:
                print(f"   ✅ Валидация: {hyp['validation']}")

            # Клиническая значимость
            if 'clinical_relevance' in hyp:
                print(f"   🏥 Клиническая значимость: {hyp['clinical_relevance']}")

            # Новизна
            if 'novelty_score' in hyp:
                print(f"   ⭐ Новизна: {hyp['novelty_score']}/10")

            # DOI источники
            doi_sources = hyp.get('doi_sources', [])
            if doi_sources:
                print(f"   📖 DOI источники:")
                for source in doi_sources:
                    print(f"      • {source['term']}: {source['doi']}")
            else:
                print(f"   📖 DOI источники: отсутствуют ⚠️")

            # Индикатор резервной гипотезы
            if hyp.get('is_fallback'):
                print(f"   ⚠️ Резервная гипотеза (ограниченные данные)")

            print("-" * 80)

    # Вспомогательные методы адаптированные под вашу структуру данных
    def _calculate_specialized_score(self, term):
        """Расчет специализированного скора"""
        if term not in self.G:
            return 0

        score = 0
        degree = self.G.degree(term)

        # Базовый скор от степени
        score += min(degree, 20)  # До 20 баллов за связи

        # Бонус за DOI
        if self._count_doi(term) > 0:
            score += 10

        # Бонус за биологические ключевые слова
        term_lower = term.lower()
        bio_keywords = ['human', 'cell', 'gene', 'protein', 'aging', 'dna', 'rna']
        for keyword in bio_keywords:
            if keyword in term_lower:
                score += 3

        return score

    def _count_doi(self, term):
        """Подсчет DOI для термина"""
        if term not in self.node_full_meta:
            return 0

        meta = self.node_full_meta[term]
        if not isinstance(meta, dict):
            return 0

        # Проверяем поле 'dois' (список)
        if 'dois' in meta:
            dois = meta['dois']
            if isinstance(dois, list):
                return len([d for d in dois if d and str(d).strip() and str(d).strip().lower() not in ['none', 'nan', 'null']])

        # Проверяем другие поля
        for field in ['doi', 'DOI', 'Doi']:
            if field in meta and meta[field]:
                return 1

        return 0

    def _get_doi(self, term):
        """Получение первого DOI для термина"""
        if term not in self.node_full_meta:
            return None

        meta = self.node_full_meta[term]
        if not isinstance(meta, dict):
            return None

        # Проверяем поле 'dois' (список)
        if 'dois' in meta:
            dois = meta['dois']
            if isinstance(dois, list):
                for doi in dois:
                    if doi and str(doi).strip() and str(doi).strip().lower() not in ['none', 'nan', 'null']:
                        return str(doi).strip()

        # Проверяем другие поля
        for field in ['doi', 'DOI', 'Doi']:
            if field in meta and meta[field]:
                return str(meta[field]).strip()

        return None
    def __init__(self, scientific_generator):
        """Оптимизированный генератор гипотез"""
        self.generator = scientific_generator
        self.hypothesis_history = {}
        self.doi_metadata = {}

    def analyze_specialized_connections(self, search_term, max_neighbors=12):
        """
        Анализ специализированных связей термина с адаптивной логикой
        """
        print(f"\n🔍 АНАЛИЗ СПЕЦИАЛИЗИРОВАННЫХ СВЯЗЕЙ: {search_term}")
        print("-" * 60)

        if search_term not in self.generator.G:
            print(f"❌ Термин '{search_term}' не найден в графе")
            return None

        # Получаем всех соседей
        all_neighbors = list(self.generator.G.neighbors(search_term))

        if len(all_neighbors) == 0:
            print(f"❌ У термина '{search_term}' нет соседей в графе")
            return None

        # Создаем селектор для оценки соседей
        selector = SpecializedTermSelector(self.generator.G, self.generator.node_full_meta)

        # Оцениваем соседей по специализированному скору
        neighbor_scores = []
        specialized_count = 0

        for neighbor in all_neighbors:
            score = selector._calculate_specialized_score(neighbor)
            if selector._matches_user_keywords(neighbor):
                specialized_count += 1
            neighbor_scores.append((neighbor, score))

        # Сортируем по скору и берем топ
        neighbor_scores.sort(key=lambda x: x[1], reverse=True)

        # АДАПТИВНАЯ ЛОГИКА: если мало специализированных соседей, берем лучших по скору
        if specialized_count < 3:
            print(f"⚠️ Найдено только {specialized_count} специализированных соседей")
            print(f"💡 Используем {max_neighbors} лучших соседей по скору")
            specialized_neighbors = [n for n, _ in neighbor_scores[:max_neighbors]]
        else:
            # Берем только соседей, соответствующих ключевым словам
            specialized_neighbors = []
            for neighbor, score in neighbor_scores:
                if len(specialized_neighbors) >= max_neighbors:
                    break
                if selector._matches_user_keywords(neighbor):
                    specialized_neighbors.append(neighbor)

        print(f"🎯 Выбрано {len(specialized_neighbors)} соседей для анализа из {len(all_neighbors)}")
        print(f"   Из них соответствуют ключевым словам: {specialized_count}")

        # Анализируем DOI
        doi_info = self._analyze_doi_metadata(specialized_neighbors)

        # КРИТИЧЕСКАЯ ПРОВЕРКА: достаточно ли соседей для анализа
        min_threshold = 1  # Снижаем порог
        sufficient_neighbors = len(specialized_neighbors) >= min_threshold

        if not sufficient_neighbors:
            print(f"⚠️ ВНИМАНИЕ: Найдено только {len(specialized_neighbors)} соседей!")
            print(f"   Это может привести к менее качественным гипотезам.")

        # Показываем информацию о соседях
        if len(specialized_neighbors) > 0:
            print(f"\n📋 Топ-5 выбранных соседей:")
            for i, neighbor in enumerate(specialized_neighbors[:5], 1):
                degree = self.generator.G.degree(neighbor)
                matches = selector._matches_user_keywords(neighbor)
                score = next((s for n, s in neighbor_scores if n == neighbor), 0)
                status = "✅" if matches else "⭐"
                print(f"   {i}. {neighbor} ({status}, связей: {degree}, скор: {score:.1f})")

        print(f"📚 DOI найдены для {len(doi_info)} из {len(specialized_neighbors)} терминов")

        # КРИТИЧЕСКАЯ ПРОВЕРКА: есть ли DOI
        has_doi = len(doi_info) > 0
        if not has_doi:
            print(f"⚠️ ВНИМАНИЕ: DOI/PMID не найдены для выбранных терминов!")
            print(f"   Гипотезы будут генерироваться без прямых научных источников.")

        return {
            'specialized_neighbors': specialized_neighbors,
            'doi_metadata': doi_info,
            'total_neighbors': len(all_neighbors),
            'neighbor_scores': neighbor_scores[:max_neighbors],
            'quality_check': {
                'sufficient_neighbors': sufficient_neighbors,
                'has_doi': has_doi,
                'neighbor_count': len(specialized_neighbors),
                'doi_count': len(doi_info),
                'specialized_count': specialized_count
            }
        }

    def _analyze_doi_metadata(self, terms):
        """Анализ DOI метаданных для терминов с учетом вашей структуры"""
        doi_info = {}

        doi_fields_to_check = ['dois', 'doi', 'DOI', 'Doi', 'pmid', 'PMID', 'pubmed_id', 'publication_id']

        for term in terms:
            if term in self.generator.node_full_meta:
                meta = self.generator.node_full_meta[term]
                if isinstance(meta, dict):
                    # Проверяем разные поля с DOI
                    doi = None
                    for field in doi_fields_to_check:
                        if field in meta:
                            field_value = meta[field]

                            # Если это список (как в вашей структуре)
                            if isinstance(field_value, list):
                                # Берем первый непустой DOI из списка
                                for doi_item in field_value:
                                    if doi_item and str(doi_item).strip() and str(doi_item).strip().lower() not in ['none', 'nan', 'null']:
                                        doi = str(doi_item).strip()
                                        break
                                if doi:
                                    break

                            # Если это строка
                            elif isinstance(field_value, str) and field_value.strip() and field_value.strip().lower() not in ['none', 'nan', 'null']:
                                doi = field_value.strip()
                                break

                    if doi:
                        # Извлекаем дополнительные метаданные
                        titles = meta.get('titles', [])
                        authors = meta.get('authors', [])
                        years = meta.get('years', [])
                        keywords = meta.get('keywords', [])

                        doi_info[term] = {
                            'doi': doi,
                            'title': titles[0] if titles and titles[0] else None,
                            'authors': authors[0] if authors and authors[0] else None,
                            'year': years[0] if years and years[0] else None,
                            'keywords': keywords[0] if keywords and keywords[0] else None,
                            'type': 'scientific_paper',
                            'source': 'literature_graph',
                            'description': f"Scientific paper: {titles[0][:100] if titles and titles[0] else 'No title'}"
                        }

                elif isinstance(meta, str) and ('doi' in meta.lower() or 'pmid' in meta.lower()):
                    # Если метаданные - строка с DOI
                    doi_info[term] = {
                        'doi': meta[:50],
                        'type': 'string_metadata',
                        'source': 'embedded',
                        'description': meta[:100]
                    }

        return doi_info

    def generate_specialized_hypotheses(self, search_term, specialized_data, count=5, temperature=0.75):
        """
        Генерация гипотез с адаптивной проверкой качества
        temperature: 0.1-1.0 для контроля креативности
        """
        print(f"\n💡 ГЕНЕРАЦИЯ СПЕЦИАЛИЗИРОВАННЫХ ГИПОТЕЗ")
        print(f"🌡️ Температура: {temperature} (0.1=консервативно, 1.0=креативно)")
        print("-" * 60)

        # АДАПТИВНАЯ ПРОВЕРКА КАЧЕСТВА ДАННЫХ
        quality_check = specialized_data.get('quality_check', {})

        sufficient_neighbors = quality_check.get('sufficient_neighbors', False)
        has_doi = quality_check.get('has_doi', False)
        neighbor_count = quality_check.get('neighbor_count', 0)
        doi_count = quality_check.get('doi_count', 0)
        specialized_count = quality_check.get('specialized_count', 0)

        print(f"🔍 ПРОВЕРКА КАЧЕСТВА ДАННЫХ:")
        print(f"   • Всего соседей для анализа: {neighbor_count}")
        print(f"   • Из них специализированных: {specialized_count}")
        print(f"   • DOI источников: {doi_count}")
        print(f"   • Достаточно соседей: {'✅' if sufficient_neighbors else '⚠️'}")
        print(f"   • Есть DOI: {'✅' if has_doi else '⚠️'}")

        # АДАПТИВНАЯ СИСТЕМА ПРЕДУПРЕЖДЕНИЙ
        warnings = []
        critical_issues = []

        if neighbor_count == 0:
            critical_issues.append("❌ НЕТ соседей для анализа")
        elif neighbor_count < 3:
            warnings.append(f"⚠️ Мало соседей для анализа ({neighbor_count} < 3)")

        if specialized_count == 0:
            warnings.append("⚠️ НЕТ специализированных соседей (используем лучших по скору)")
        elif specialized_count < 2:
            warnings.append(f"⚠️ Мало специализированных соседей ({specialized_count} < 2)")

        if doi_count == 0:
            warnings.append("⚠️ НЕТ DOI источников")

        # Если есть критические проблемы, блокируем
        if critical_issues:
            print(f"\n🚨 КРИТИЧЕСКИЕ ПРОБЛЕМЫ:")
            for issue in critical_issues:
                print(f"   {issue}")
            print(f"❌ Генерация невозможна")
            return None

        # Если есть предупреждения, показываем их
        if warnings:
            print(f"\n⚠️ ПРЕДУПРЕЖДЕНИЯ О КАЧЕСТВЕ:")
            for warning in warnings:
                print(f"   {warning}")

            print(f"\n💡 АДАПТИВНЫЕ МЕРЫ:")
            if specialized_count == 0:
                print(f"   • Используем соседей с высоким скором релевантности")
            if doi_count == 0:
                print(f"   • Гипотезы будут основаны на структурных связях графа")
            if neighbor_count < 3:
                print(f"   • Генерируем {count} гипотез с ограниченным контекстом")

            # Менее строгий запрос подтверждения
            if len(warnings) >= 2:  # Только если много предупреждений
                confirm = input(f"\n❓ Продолжить генерацию с указанными ограничениями? (Y/n): ").strip().lower()

                if confirm in ['n', 'no', 'нет']:
                    print(f"❌ Генерация отменена пользователем")
                    return None
                else:
                    print(f"✅ Продолжаем с адаптивными мерами...")

        specialized_neighbors = specialized_data['specialized_neighbors']
        doi_metadata = specialized_data['doi_metadata']
        neighbor_scores = specialized_data['neighbor_scores']

        # Создаем специализированный промпт
        prompt = self._create_specialized_prompt(
            search_term, specialized_neighbors, doi_metadata, neighbor_scores, count
        )

        try:
            response = self.generator.client.chat.completions.create(
                model="llama3.1",
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature,
                max_tokens=4000
            )

            response_text = response.choices[0].message.content.strip()

            # Парсинг JSON
            try:
                hypotheses_data = json.loads(response_text)
            except json.JSONDecodeError:
                start_idx = response_text.find('{')
                end_idx = response_text.rfind('}') + 1
                if start_idx != -1 and end_idx != -1:
                    json_text = response_text[start_idx:end_idx]
                    hypotheses_data = json.loads(json_text)
                else:
                    raise ValueError("Не удалось извлечь JSON")

            # Добавляем метаданные о качестве и температуре
            hypotheses_data['generation_temperature'] = temperature
            hypotheses_data['data_quality'] = quality_check
            hypotheses_data['generation_warnings'] = warnings
            hypotheses_data['adaptive_measures'] = []

            if specialized_count == 0:
                hypotheses_data['adaptive_measures'].append("Используются соседи с высоким скором")
            if doi_count == 0:
                hypotheses_data['adaptive_measures'].append("Основано на структурных связях графа")

            # Добавляем DOI метаданные и скоры к гипотезам
            if 'hypotheses' in hypotheses_data:
                for hyp in hypotheses_data['hypotheses']:
                    hyp['doi_sources'] = []
                    hyp['term_scores'] = []

                    for term in hyp.get('terms', []):
                        if term in doi_metadata:
                            hyp['doi_sources'].append({
                                'term': term,
                                'doi': doi_metadata[term]['doi']
                            })

                        # Добавляем скоры терминов
                        for scored_term, score in neighbor_scores:
                            if scored_term == term:
                                hyp['term_scores'].append({
                                    'term': term,
                                    'score': score
                                })
                                break

            quality_level = "высокое" if not warnings else "среднее" if len(warnings) < 2 else "ограниченное"
            print(f"✅ Гипотезы сгенерированы (качество данных: {quality_level})")
            return hypotheses_data

        except Exception as e:
            print(f"❌ Ошибка генерации гипотез: {e}")
            return None

    def _create_specialized_prompt(self, search_term, specialized_neighbors, doi_metadata, neighbor_scores, count):
        """Создание специализированного промпта с высокой конкретикой"""

        # Формируем информацию о соседях с скорами
        neighbors_info = "СПЕЦИАЛИЗИРОВАННЫЕ СВЯЗАННЫЕ ТЕРМИНЫ (с приоритетом):\n"
        for i, (term, score) in enumerate(neighbor_scores[:15], 1):
            neighbors_info += f"{i}. {term} (приоритет: {score:.1f})\n"

        # Формируем информацию о DOI
        doi_info = ""
        if doi_metadata:
            doi_info = "\nДОСТУПНЫЕ DOI ИСТОЧНИКИ:\n"
            for term, meta in doi_metadata.items():
                doi_info += f"- {term}: DOI {meta['doi']}\n"

        return f"""Ты — эксперт по молекулярной биологии старения человека. Создавай МАКСИМАЛЬНО КОНКРЕТНЫЕ и ИЗМЕРИМЫЕ гипотезы.

ОСНОВНОЙ ТЕРМИН: "{search_term}"

{neighbors_info}

{doi_info}

КРИТИЧЕСКИ ВАЖНО - КАЖДАЯ ГИПОТЕЗА ДОЛЖНА ВКЛЮЧАТЬ:

🧬 ЭПИГЕНЕТИЧЕСКАЯ КОНКРЕТИКА:
- Конкретные CpG-сайты (например, cg05575921, cg09809672)
- Гистоновые метки (H3K4me3, H3K27me3, H3K9me3, H4K16ac)
- Конкретные DNMT (DNMT1, DNMT3A, DNMT3B)
- TET-ферменты (TET1, TET2, TET3)

🔥 ВОСПАЛИТЕЛЬНАЯ КОНКРЕТИКА:
- Конкретные цитокины (IL-1β, IL-6, TNF-α, IL-8, MCP-1)
- Рецепторы (TLR4, TLR2, IL-1R, TNFR1/2)
- Сигнальные пути (NF-κB p65, IRF3, STAT3)
- Inflammasome компоненты (NLRP3, ASC, caspase-1)

🧪 ИММУННАЯ КОНКРЕТИКА:
- Субпопуляции клеток (CD4+CD28-, CD8+CD57+, Tregs CD4+CD25+FOXP3+)
- Поверхностные маркеры (CD38, CD57, KLRG1, PD-1, LAG-3)
- Секретируемые факторы (IFN-γ, perforin, granzyme B)

📊 ЭКСПЕРИМЕНТАЛЬНАЯ КОНКРЕТИКА:
- In vitro модели: первичные фибробласты человека, HUVEC, PBMC
- In vivo модели: мыши C57BL/6, крысы Wistar, приматы
- Клинические когорты: возраст участников, размер выборки
- Конкретные методы: qPCR праймеры, антитела для Western blot, ELISA киты

💉 БИОМАРКЕРЫ - УКАЖИ ТОЧНО:
- Какие белки измерять в плазме/сыворотке (нг/мл, пг/мл)
- Какие мРНК в каких тканях (fold change, ΔCt)
- Какие метилированные сайты (% methylation)
- Функциональные тесты (время, концентрации)

ЗАДАЧА: Сгенерируй {count} МАКСИМАЛЬНО КОНКРЕТНЫХ гипотез о "{search_term}" в старении человека.

ФОРМАТ JSON:
{{
  "focus": "конкретные молекулярные механизмы с измеримыми параметрами",
  "search_term": "{search_term}",
  "hypotheses": [
    {{
      "terms": ["{search_term}", "конкретный_термин1", "конкретный_термин2"],
      "hypothesis": "Конкретная гипотеза с точными молекулярными мишенями",
      "rationale": "Механистическое обоснование с конкретными белками/генами",
      "epigenetic_specifics": "Конкретные CpG-сайты, гистоновые метки, ферменты",
      "inflammatory_specifics": "Точные цитокины, рецепторы, концентрации",
      "immune_specifics": "Конкретные клеточные популяции и маркеры",
      "experimental_design": "Детальный протокол: модель, методы, временные точки",
      "measurable_biomarkers": "Конкретные измеримые параметры с единицами",
      "clinical_validation": "Как проверить у человека: когорта, методы, биомаркеры",
      "therapeutic_target": "Конкретная мишень для вмешательства"
    }}
  ]
}}

ПРИМЕРЫ КОНКРЕТИКИ:
❌ ПЛОХО: "изменения метилирования ДНК"
✅ ХОРОШО: "гиперметилирование CpG-сайта cg05575921 в промотере CDKN2A"

❌ ПЛОХО: "провоспалительные цитокины"
✅ ХОРОШО: "повышение IL-6 до 15.2±3.1 пг/мл и TNF-α до 8.7±2.3 пг/мл в плазме"

❌ ПЛОХО: "иммунные клетки"
✅ ХОРОШО: "увеличение CD8+CD57+KLRG1+ сенесцентных T-клеток до 23±5% от CD8+"

Генерируй ТОЛЬКО JSON с максимальной конкретикой!"""

    def display_specialized_hypotheses(self, hypotheses_data):
        """Показ специализированных гипотез с конкретными метаданными"""
        if not hypotheses_data or 'hypotheses' not in hypotheses_data:
            print("❌ Нет данных для отображения")
            return

        hypotheses = hypotheses_data['hypotheses']
        temperature = hypotheses_data.get('generation_temperature', 'неизвестно')
        data_quality = hypotheses_data.get('data_quality', {})
        warnings = hypotheses_data.get('generation_warnings', [])

        print(f"\n📚 КОНКРЕТНЫЕ ГИПОТЕЗЫ СТАРЕНИЯ ЧЕЛОВЕКА")
        print(f"🌡️ Температура генерации: {temperature}")

        # Показываем предупреждения о качестве
        if warnings:
            print(f"⚠️ ПРЕДУПРЕЖДЕНИЯ О КАЧЕСТВЕ:")
            for warning in warnings:
                print(f"   {warning}")
            print(f"🔍 Качество данных: специализированных соседей={data_quality.get('neighbor_count', 0)}, DOI={data_quality.get('doi_count', 0)}")
        else:
            print(f"✅ Качество данных: специализированных соседей={data_quality.get('neighbor_count', 0)}, DOI={data_quality.get('doi_count', 0)}")

        print("=" * 80)

        for i, hyp in enumerate(hypotheses, 1):
            print(f"\n{i}. ГИПОТЕЗА:")
            print(f"   🧬 Термины: {', '.join(hyp.get('terms', []))}")
            print(f"   🔬 Механизм: {hyp.get('hypothesis', '')}")
            print(f"   📚 Обоснование: {hyp.get('rationale', '')}")

            # Новые конкретные поля
            if 'epigenetic_specifics' in hyp:
                print(f"   🧬 Эпигенетика: {hyp['epigenetic_specifics']}")

            if 'inflammatory_specifics' in hyp:
                print(f"   🔥 Воспаление: {hyp['inflammatory_specifics']}")

            if 'immune_specifics' in hyp:
                print(f"   🛡️  Иммунитет: {hyp['immune_specifics']}")

            if 'experimental_design' in hyp:
                print(f"   🧪 Эксперимент: {hyp['experimental_design']}")

            if 'measurable_biomarkers' in hyp:
                print(f"   📊 Биомаркеры: {hyp['measurable_biomarkers']}")

            if 'clinical_validation' in hyp:
                print(f"   🏥 Клиническая валидация: {hyp['clinical_validation']}")

            if 'therapeutic_target' in hyp:
                print(f"   💊 Терапевтическая мишень: {hyp['therapeutic_target']}")

            # Старые поля для совместимости
            if 'clinical_relevance' in hyp:
                print(f"   🎯 Клиническая значимость: {hyp['clinical_relevance']}")

            # Скоры терминов
            if hyp.get('term_scores'):
                print(f"   📈 Приоритет терминов:")
                for term_score in hyp['term_scores']:
                    print(f"      • {term_score['term']}: {term_score['score']:.1f}")

            # DOI источники
            if hyp.get('doi_sources'):
                print(f"   📖 DOI источники:")
                for source in hyp['doi_sources']:
                    print(f"      • {source['term']}: {source['doi']}")
            else:
                print(f"   📖 DOI источники: отсутствуют ⚠️")

            print("-" * 80)

        # Итоговая оценка качества результата
        if warnings:
            print(f"\n⚠️ ИТОГОВАЯ ОЦЕНКА: Гипотезы сгенерированы с ограничениями")
            print(f"💡 Рекомендуется выбрать термин с большим количеством релевантных связей и DOI")
        else:
            print(f"\n✅ ИТОГОВАЯ ОЦЕНКА: Гипотезы высокого качества с научным обоснованием")

def specialized_interactive_mode(graph=None, node_meta=None, edge_meta=None):
    """
    Специализированный интерактивный режим на основе ваших ключевых слов
    """
    print("🚀 СПЕЦИАЛИЗИРОВАННЫЙ РЕЖИМ НА ОСНОВЕ ВАШИХ КЛЮЧЕВЫХ СЛОВ")
    print("=" * 80)
    print("Специализация:")
    print("  🧬 Биомаркеры старения человека")
    print("  🔬 Эпигенетические часы")
    print("  ⚡ Воспаление при старении")
    print("  🧪 Специфичные гены (ATAD3A, COL18A1, GATA4, etc.)")
    print("  🏥 Терапевтические мишени")
    print("  🧠 Иммунное старение")
    print("  🔀 Пути долголетия")
    print("  📊 Одноклеточный анализ")
    print("=" * 80)

    # Пытаемся найти граф в разных местах
    G_to_use = None
    node_meta_to_use = None
    edge_meta_to_use = None

    if graph is not None and node_meta is not None:
        G_to_use = graph
        node_meta_to_use = node_meta
        edge_meta_to_use = edge_meta
    else:
        # Пытаемся найти в глобальных переменных
        try:
            import inspect
            frame = inspect.currentframe()
            while frame:
                if 'G' in frame.f_globals and 'node_full_meta' in frame.f_globals:
                    G_to_use = frame.f_globals['G']
                    node_meta_to_use = frame.f_globals['node_full_meta']
                    edge_meta_to_use = frame.f_globals.get('edge_full_meta')
                    break
                frame = frame.f_back
        except:
            pass

    if G_to_use is None:
        print("❌ Граф не найден!")
        print("💡 Используйте: specialized_interactive_mode(G, node_full_meta, edge_full_meta)")
        print("   где G, node_full_meta, edge_full_meta - ваши переменные")
        return

    try:
        # Создаем специализированные компоненты
        print(f"📊 Используем граф: {G_to_use.number_of_nodes():,} узлов, {G_to_use.number_of_edges():,} рёбер")

        specialized_selector = SpecializedTermSelector(G_to_use, node_meta_to_use)

        from openai import OpenAI
        client = OpenAI(base_url="http://80.209.242.40:8000/v1", api_key="dummy-key")

        # Создаем временный scientific_generator для совместимости
        class TempScientificGenerator:
            def __init__(self, G, node_meta, edge_meta, client):
                self.G = G
                self.node_full_meta = node_meta
                self.edge_full_meta = edge_meta
                self.client = client

        scientific_generator = TempScientificGenerator(G_to_use, node_meta_to_use, edge_meta_to_use, client)
        specialized_generator = OptimizedHypothesisGenerator(scientific_generator)

    except Exception as e:
        print(f"❌ Ошибка создания компонентов: {e}")
        return

    current_term = None

    while True:
        try:
            if current_term is None:
                print(f"\n{'🎯' * 20}")
                print("ВЫБОР СПЕЦИАЛИЗИРОВАННОГО ТЕРМИНА")
                print(f"{'🎯' * 20}")

                specialized_selector.show_categories_specialized()

                choice = input("\n👆 Выберите категорию (1-8), 0 для ручного ввода, 99 для случайного: ").strip()

                if choice == '0':
                    # Ручной ввод с валидацией
                    manual_term = input("✍️ Введите термин: ").strip()
                    if manual_term:
                        is_valid, message, quality_info = specialized_selector.validate_manual_term(manual_term)

                        print(f"\n🔍 ВАЛИДАЦИЯ ТЕРМИНА '{manual_term}':")
                        print(message)

                        if quality_info:
                            print(f"\n📊 ХАРАКТЕРИСТИКИ:")
                            print(f"   🔗 Всего связей: {quality_info['degree']}")
                            print(f"   🎯 Специализированных соседей: {quality_info['specialized_neighbors']}")
                            print(f"   🔑 Соответствует ключевым словам: {'✅' if quality_info['matches_keywords'] else '❌'}")
                            print(f"   📚 DOI: {'✅' if quality_info['has_doi'] else '❌'}")
                            print(f"   📈 Специализированный скор: {quality_info['spec_score']:.1f}")

                        if not is_valid:
                            confirm = input(f"\n❓ Продолжить с этим термином несмотря на низкое качество? (y/N): ").strip().lower()
                            if confirm not in ['y', 'yes', 'да', 'д']:
                                print("❌ Выбор термина отменен")
                                continue

                        current_term = manual_term
                        if is_valid:
                            print(f"✅ Термин принят для анализа")
                        else:
                            print(f"⚠️ Термин принят с предупреждениями")

                        specialized_selector.show_term_details(current_term)
                    continue

                elif choice == '99':
                    current_term = specialized_selector.get_random_term()
                    if current_term:
                        print(f"🎲 Выбран случайный качественный термин: {current_term}")

                        # Показываем почему этот термин качественный
                        spec_neighbors = specialized_selector._count_specialized_neighbors(current_term)
                        doi_status = "📚" if current_term in specialized_selector._doi_cache else "❌"
                        score = specialized_selector._calculate_specialized_score(current_term)

                        print(f"✅ КАЧЕСТВЕННЫЕ ПОКАЗАТЕЛИ:")
                        print(f"   🎯 Специализированных соседей: {spec_neighbors} (≥3 ✅)")
                        print(f"   📚 DOI: {doi_status}")
                        print(f"   📈 Специализированный скор: {score:.1f}")

                        specialized_selector.show_term_details(current_term)
                    else:
                        print("❌ Не удалось найти качественный термин")
                        print("💡 Попробуйте расширить ключевые слова или проверить данные")
                    continue

                elif choice.lower() in ['quit', 'exit', 'q']:
                    break

                else:
                    try:
                        category_num = int(choice)
                        category_name, terms = specialized_selector.get_category_terms(category_num)

                        if terms:
                            print(f"\n📋 {category_name}")
                            print("=" * 60)

                            for i, term in enumerate(terms, 1):
                                metadata = specialized_selector.get_term_metadata(term)
                                doi_status = "📚" if metadata['doi'] else "❌"
                                keywords_count = len(metadata['matches_keywords'])
                                print(f"{i:2}. {term:<35} (связей: {metadata['degree']}, DOI: {doi_status}, КС: {keywords_count}, скор: {metadata['specialized_score']:.1f})")

                            term_choice = input(f"\n👆 Выберите термин (1-{len(terms)}): ").strip()
                            term_idx = int(term_choice) - 1

                            if 0 <= term_idx < len(terms):
                                current_term = terms[term_idx]
                                specialized_selector.show_term_details(current_term)

                    except (ValueError, IndexError):
                        print("⚠️ Неверный выбор")
                        continue

            else:
                # Работа с выбранным термином
                print(f"\n{'🔬' * 30}")
                print(f"СПЕЦИАЛИЗИРОВАННЫЙ АНАЛИЗ: {current_term.upper()}")
                print(f"{'🔬' * 30}")

                print("Выберите действие:")
                print("1. 🔍 Анализ специализированных связей")
                print("2. 💡 Генерация клинических гипотез")
                print("3. 📚 Показать метаданные и DOI")
                print("4. 🔄 Выбрать новый термин")
                print("5. ❌ Выход")

                action = input("\n👆 Выберите действие (1-5): ").strip()

                if action == '1':
                    print("⏳ Анализ специализированных связей...")
                    spec_data = specialized_generator.analyze_specialized_connections(current_term)

                    if spec_data:
                        quality_check = spec_data.get('quality_check', {})

                        print(f"\n📊 РЕЗУЛЬТАТЫ АНАЛИЗА:")
                        print(f"✅ Найдено {len(spec_data['specialized_neighbors'])} специализированных связей")
                        print(f"📚 DOI доступны для {len(spec_data['doi_metadata'])} терминов")

                        # Оценка качества
                        if quality_check.get('sufficient_neighbors') and quality_check.get('has_doi'):
                            print(f"🌟 КАЧЕСТВО: Отличное - достаточно связей и есть DOI")
                        elif quality_check.get('sufficient_neighbors'):
                            print(f"⚠️ КАЧЕСТВО: Хорошее - достаточно связей, но мало DOI")
                        elif quality_check.get('has_doi'):
                            print(f"⚠️ КАЧЕСТВО: Среднее - есть DOI, но мало связей")
                        else:
                            print(f"❌ КАЧЕСТВО: Низкое - мало связей и нет DOI")

                        # Показываем топ связи
                        if len(spec_data['neighbor_scores']) > 0:
                            print(f"\n🔗 Топ-5 специализированных связей:")
                            for i, (neighbor, score) in enumerate(spec_data['neighbor_scores'][:5], 1):
                                doi_status = "📚" if neighbor in spec_data['doi_metadata'] else "❌"
                                print(f"   {i}. {neighbor} (скор: {score:.1f}, DOI: {doi_status})")

                        # Рекомендации
                        if not quality_check.get('sufficient_neighbors'):
                            print(f"\n💡 РЕКОМЕНДАЦИЯ: Выберите термин с большим количеством релевантных связей")
                        if not quality_check.get('has_doi'):
                            print(f"💡 РЕКОМЕНДАЦИЯ: Проверьте наличие DOI/PMID в метаданных")
                    else:
                        print(f"❌ Анализ не удался")

                elif action == '2':
                    print("⏳ Генерация конкретных клинических гипотез...")

                    # Запрашиваем температуру у пользователя
                    temp_input = input("🌡️ Выберите температуру (0.1-1.0) [по умолчанию 0.75]: ").strip()
                    try:
                        if temp_input:
                            temperature = float(temp_input)
                            if not (0.1 <= temperature <= 1.0):
                                print("⚠️ Температура должна быть от 0.1 до 1.0, используем 0.75")
                                temperature = 0.75
                        else:
                            temperature = 0.75
                    except ValueError:
                        print("⚠️ Неверный формат, используем температуру 0.75")
                        temperature = 0.75

                    print(f"🌡️ Используем температуру: {temperature}")
                    print("💡 Совет: 0.1-0.3=консервативно, 0.4-0.7=сбалансированно, 0.8-1.0=креативно")

                    spec_data = specialized_generator.analyze_specialized_connections(current_term)

                    if spec_data:
                        hypotheses = specialized_generator.generate_specialized_hypotheses(
                            current_term, spec_data, 5, temperature
                        )

                        if hypotheses:
                            specialized_generator.display_specialized_hypotheses(hypotheses)

                elif action == '3':
                    metadata = specialized_selector.get_term_metadata(current_term)
                    print(f"\n📚 МЕТАДАННЫЕ ДЛЯ '{current_term}':")
                    print(f"DOI: {metadata['doi'] or 'не найден'}")
                    print(f"Ключевые слова: {', '.join(metadata['matches_keywords'][:10])}")
                    print(f"Специализированный скор: {metadata['specialized_score']:.1f}")

                elif action == '4':
                    current_term = None
                    continue

                elif action == '5':
                    break

                else:
                    print("⚠️ Неверный выбор")

        except KeyboardInterrupt:
            print("\n⏹️ Прерывание (Ctrl+C)")
            break
        except Exception as e:
            print(f"❌ Ошибка: {e}")
            continue

# === ДИАГНОСТИЧЕСКАЯ ФУНКЦИЯ ===
# === ДИАГНОСТИЧЕСКИЕ ФУНКЦИИ ===
def explore_graph_terms(G, node_full_meta, top_n=50):
    """
    Исследование реальных терминов в графе для настройки системы
    """
    print("🔍 ИССЛЕДОВАНИЕ ТЕРМИНОВ В ВАШЕМ ГРАФЕ")
    print("=" * 70)

    if not G or G.number_of_nodes() == 0:
        print("❌ Граф пуст")
        return

    print(f"📊 Всего узлов в графе: {G.number_of_nodes():,}")
    print(f"📊 Всего рёбер в графе: {G.number_of_edges():,}")

    # Получаем все узлы с их степенями
    degree_cache = dict(G.degree())
    terms_by_degree = [(term, degree) for term, degree in degree_cache.items()]
    terms_by_degree.sort(key=lambda x: x[1], reverse=True)

    print(f"\n🔝 ТОП-{top_n} ТЕРМИНОВ ПО КОЛИЧЕСТВУ СВЯЗЕЙ:")
    print("=" * 70)

    for i, (term, degree) in enumerate(terms_by_degree[:top_n], 1):
        # Проверяем наличие DOI
        doi_status = "📚" if term in node_full_meta and _has_doi(node_full_meta[term]) else "❌"
        print(f"{i:2}. {term:<45} (связей: {degree:3}, DOI: {doi_status})")

    # Анализируем по словам
    print(f"\n🔍 АНАЛИЗ ПО БИОЛОГИЧЕСКИМ СЛОВАМ:")
    print("=" * 70)

    bio_words = {
        'human': [], 'cell': [], 'gene': [], 'protein': [], 'dna': [], 'aging': [],
        'inflammation': [], 'immune': [], 'therapy': [], 'methylation': [],
        'expression': [], 'signaling': [], 'pathway': [], 'cancer': [], 'diabetes': []
    }

    # Ищем термины, содержащие биологические слова
    for term, degree in terms_by_degree[:200]:  # Проверяем топ-200
        term_lower = term.lower()
        for bio_word in bio_words:
            if bio_word in term_lower and degree >= 3:  # Минимум 3 связи
                bio_words[bio_word].append((term, degree))

    # Показываем результаты
    for bio_word, terms in bio_words.items():
        if terms:
            terms.sort(key=lambda x: x[1], reverse=True)  # Сортируем по степени
            top_terms = terms[:5]  # Топ-5
            print(f"\n🔬 '{bio_word}' ({len(terms)} терминов):")
            for i, (term, degree) in enumerate(top_terms, 1):
                doi_status = "📚" if term in node_full_meta and _has_doi(node_full_meta[term]) else "❌"
                print(f"   {i}. {term:<40} (связей: {degree}, DOI: {doi_status})")

    print(f"\n💡 РЕКОМЕНДАЦИИ ДЛЯ НАСТРОЙКИ:")
    print("1. Используйте термины из списка выше для анализа")
    print("2. Выберите термин с высокой степенью связности (>10)")
    print("3. Предпочтительно с DOI для научного обоснования")
    print("4. Или запустите режим 'без фильтрации' для работы с любыми терминами")

def _has_doi(meta):
    """Проверка наличия DOI в метаданных"""
    if isinstance(meta, dict):
        doi_fields = ['dois', 'doi', 'DOI', 'Doi']
        for field in doi_fields:
            if field in meta:
                value = meta[field]
                if isinstance(value, list):
                    return any(d and str(d).strip() and str(d).strip().lower() not in ['none', 'nan', 'null'] for d in value)
                elif value and str(value).strip() and str(value).strip().lower() not in ['none', 'nan', 'null']:
                    return True
    return False

def create_custom_categories(G, node_full_meta, min_degree=5):
    """
    Создание категорий на основе реальных данных в графе
    """
    print("🏗️ СОЗДАНИЕ КАТЕГОРИЙ НА ОСНОВЕ РЕАЛЬНЫХ ДАННЫХ")
    print("=" * 70)

    # Получаем популярные термины
    degree_cache = dict(G.degree())
    popular_terms = [(term, degree) for term, degree in degree_cache.items() if degree >= min_degree]
    popular_terms.sort(key=lambda x: x[1], reverse=True)

    print(f"📊 Найдено {len(popular_terms)} терминов с ≥{min_degree} связями")

    # Автоматически группируем по общим словам
    auto_categories = {
        '🧬 Человек и возраст': {
            'patterns': [r'human', r'age', r'aging', r'old', r'adult', r'elderly'],
            'terms': []
        },
        '🔬 Клетки и биология': {
            'patterns': [r'cell', r'cellular', r'tissue', r'organ', r'biological'],
            'terms': []
        },
        '🧪 Гены и белки': {
            'patterns': [r'gene', r'protein', r'enzyme', r'receptor', r'expression'],
            'terms': []
        },
        '⚡ Болезни и патология': {
            'patterns': [r'cancer', r'diabetes', r'disease', r'disorder', r'pathology'],
            'terms': []
        },
        '🔥 Иммунитет и воспаление': {
            'patterns': [r'immune', r'inflammation', r'inflammatory', r'cytokine'],
            'terms': []
        },
        '💊 Лечение и терапия': {
            'patterns': [r'therapy', r'treatment', r'drug', r'therapeutic', r'clinical'],
            'terms': []
        },
        '📊 Молекулярные процессы': {
            'patterns': [r'dna', r'rna', r'methylation', r'signaling', r'pathway'],
            'terms': []
        },
        '🎯 Высокосвязанные термины': {
            'patterns': [],  # Для самых популярных терминов
            'terms': []
        }
    }

    # Распределяем термины
    unmatched_terms = []

    for term, degree in popular_terms[:200]:  # Анализируем топ-200
        term_lower = term.lower()
        matched = False

        # Проверяем паттерны для каждой категории
        for category, data in list(auto_categories.items())[:-1]:  # Исключаем последнюю категорию
            for pattern in data['patterns']:
                if re.search(pattern, term_lower):
                    data['terms'].append(term)
                    matched = True
                    break
            if matched:
                break

        if not matched:
            unmatched_terms.append((term, degree))

    # Добавляем самые популярные неопознанные термины в последнюю категорию
    auto_categories['🎯 Высокосвязанные термины']['terms'] = [
        term for term, _ in unmatched_terms[:15]
    ]

    # Ограничиваем количество терминов в каждой категории
    for category, data in auto_categories.items():
        if len(data['terms']) > 15:
            # Сортируем по степени и берем топ-15
            terms_with_degree = [(term, degree_cache.get(term, 0)) for term in data['terms']]
            terms_with_degree.sort(key=lambda x: x[1], reverse=True)
            data['terms'] = [term for term, _ in terms_with_degree[:15]]

    # Показываем результаты
    print(f"\n📋 АВТОМАТИЧЕСКИ СОЗДАННЫЕ КАТЕГОРИИ:")
    total_terms = 0
    for category, data in auto_categories.items():
        terms_count = len(data['terms'])
        total_terms += terms_count
        print(f"\n{category} ({terms_count} терминов):")

        if terms_count > 0:
            for i, term in enumerate(data['terms'][:5], 1):  # Показываем топ-5
                degree = degree_cache.get(term, 0)
                doi_status = "📚" if _has_doi(node_full_meta.get(term, {})) else "❌"
                print(f"   {i}. {term:<35} (связей: {degree}, DOI: {doi_status})")
            if terms_count > 5:
                print(f"   ... и еще {terms_count - 5} терминов")
        else:
            print("   Термины не найдены")

    print(f"\n✅ Всего подготовлено {total_terms} терминов для анализа")
    return auto_categories

def enhanced_no_filter_mode(G, node_full_meta, edge_full_meta):
    """
    Расширенный режим без фильтрации с улучшенным генератором гипотез
    """
    print("🚀 РАСШИРЕННЫЙ РЕЖИМ БЕЗ ФИЛЬТРАЦИИ")
    print("=" * 70)
    print("Возможности:")
    print("• 💡 Умная генерация гипотез с контролем качества")
    print("• 🔧 Адаптивные промпты под ваши данные")
    print("• 📊 Детальная оценка качества терминов")
    print("• 🧪 Конкретные измеримые параметры")
    print("• 📚 Интеграция DOI источников")
    print("• ✅ Резервные механизмы для любых данных")
    print("=" * 70)

    try:
        from openai import OpenAI
        client = OpenAI(base_url="http://80.209.242.40:8000/v1", api_key="dummy-key")

        # Создаем улучшенный генератор
        enhanced_generator = EnhancedHypothesisGenerator(G, node_full_meta, edge_full_meta, client)

    except Exception as e:
        print(f"❌ Ошибка создания компонентов: {e}")
        return

    current_term = None

    while True:
        try:
            if current_term is None:
                print(f"\n{'🎯' * 20}")
                print("ВЫБОР ТЕРМИНА ДЛЯ УЛУЧШЕННОГО АНАЛИЗА")
                print(f"{'🎯' * 20}")

                print("Варианты:")
                print("1. 🔍 Исследовать граф (популярные термины)")
                print("2. ✍️ Ввести термин вручную")
                print("3. 🎲 Случайный качественный термин")
                print("4. 🏆 Рекомендуемые термины (на основе ваших данных)")
                print("5. ❌ Выход")

                choice = input("\n👆 Выберите действие (1-5): ").strip()

                if choice == '1':
                    explore_graph_terms(G, node_full_meta, 30)
                    continue

                elif choice == '2':
                    manual_term = input("✍️ Введите термин: ").strip()
                    if manual_term and manual_term in G:
                        current_term = manual_term

                        # Показываем мгновенную оценку
                        quality_info = enhanced_generator.assess_term_quality(manual_term)
                        enhanced_generator._display_quality_info(quality_info)
                    else:
                        print(f"❌ Термин '{manual_term}' не найден в графе")
                    continue

                elif choice == '3':
                    degree_cache = dict(G.degree())
                    # Ищем термины с DOI и хорошими связями
                    good_terms = []
                    for term, degree in degree_cache.items():
                        if degree >= 3 and enhanced_generator._count_doi(term) > 0:
                            good_terms.append((term, degree, enhanced_generator._count_doi(term)))

                    if good_terms:
                        # Сортируем по комбинированному скору
                        good_terms.sort(key=lambda x: x[1] + x[2]*5, reverse=True)
                        current_term = random.choice(good_terms[:20])[0]  # Из топ-20

                        print(f"🎲 Выбран качественный термин: {current_term}")
                        quality_info = enhanced_generator.assess_term_quality(current_term)
                        enhanced_generator._display_quality_info(quality_info)
                    else:
                        # Резервный выбор
                        popular_terms = [(term, degree) for term, degree in degree_cache.items() if degree >= 5]
                        if popular_terms:
                            current_term = random.choice(popular_terms)[0]
                            print(f"🎲 Выбран популярный термин: {current_term}")
                        else:
                            print("❌ Подходящие термины не найдены")
                    continue

                elif choice == '4':
                    # Показываем рекомендуемые термины на основе анализа их данных
                    print("\n🏆 РЕКОМЕНДУЕМЫЕ ТЕРМИНЫ НА ОСНОВЕ ВАШИХ ДАННЫХ:")
                    print("(высокое качество: много связей + DOI)")

                    degree_cache = dict(G.degree())
                    recommendations = []

                    for term, degree in degree_cache.items():
                        doi_count = enhanced_generator._count_doi(term)
                        if degree >= 5 and doi_count > 0:
                            spec_score = enhanced_generator._calculate_specialized_score(term)
                            recommendations.append((term, degree, doi_count, spec_score))

                    # Сортируем по качеству
                    recommendations.sort(key=lambda x: x[1] + x[2]*5 + x[3], reverse=True)

                    print(f"\nТоп-10 рекомендованных терминов:")
                    for i, (term, degree, doi_count, spec_score) in enumerate(recommendations[:10], 1):
                        print(f"{i:2}. {term:<35} (связей: {degree}, DOI: {doi_count}, скор: {spec_score:.1f})")

                    if recommendations:
                        term_choice = input(f"\n👆 Выберите термин (1-{min(10, len(recommendations))}) или 0 для возврата: ").strip()
                        try:
                            term_idx = int(term_choice) - 1
                            if 0 <= term_idx < min(10, len(recommendations)):
                                current_term = recommendations[term_idx][0]
                                print(f"✅ Выбран: {current_term}")
                        except (ValueError, IndexError):
                            if term_choice != '0':
                                print("⚠️ Неверный выбор")
                    continue

                elif choice == '5':
                    break

                else:
                    print("⚠️ Неверный выбор")
                    continue

            else:
                # Работа с выбранным термином
                print(f"\n{'🔬' * 40}")
                print(f"УЛУЧШЕННЫЙ АНАЛИЗ: {current_term.upper()}")
                print(f"{'🔬' * 40}")

                print("Выберите действие:")
                print("1. 📊 Оценка качества термина")
                print("2. 🔍 Анализ связей и контекста")
                print("3. 💡 Генерация гипотез (стандарт)")
                print("4. 🔥 Генерация гипотез (высокая креативность)")
                print("5. ❄️ Генерация гипотез (консервативно)")
                print("6. 🎛️ Кастомная генерация (выбор параметров)")
                print("7. 🔄 Выбрать новый термин")
                print("8. ❌ Выход")

                action = input("\n👆 Выберите действие (1-8): ").strip()

                if action == '1':
                    print("⏳ Оценка качества термина...")
                    quality_info = enhanced_generator.assess_term_quality(current_term)
                    enhanced_generator._display_quality_info(quality_info)

                    # Дополнительная информация
                    connections = enhanced_generator.analyze_connections(current_term)
                    print(f"\n🔗 АНАЛИЗ СВЯЗЕЙ:")
                    print(f"   Всего соседей: {connections['total_neighbors']}")
                    print(f"   С DOI: {connections['high_quality_neighbors']}")
                    print(f"   Средняя степень соседей: {connections['avg_degree']:.1f}")

                elif action == '2':
                    print("⏳ Детальный анализ связей...")
                    connections = enhanced_generator.analyze_connections(current_term)

                    print(f"\n🔗 ДЕТАЛЬНЫЙ АНАЛИЗ СВЯЗЕЙ ДЛЯ '{current_term}':")
                    print(f"   Всего соседей: {connections['total_neighbors']}")
                    print(f"   С DOI источниками: {connections['high_quality_neighbors']}")

                    if connections['neighbors']:
                        print(f"\n🏆 Топ-10 связанных терминов:")
                        for i, neighbor in enumerate(connections['neighbors'][:10], 1):
                            doi_status = "📚" if neighbor['has_doi'] else "❌"
                            print(f"   {i:2}. {neighbor['term']:<30} (связей: {neighbor['degree']}, DOI: {doi_status}, скор: {neighbor['spec_score']:.1f})")

                elif action == '3':
                    print("⏳ Стандартная генерация гипотез...")
                    result = enhanced_generator.generate_hypotheses(current_term, count=5, temperature=0.7)
                    if result:
                        enhanced_generator.display_hypotheses(result)

                elif action == '4':
                    print("⏳ Креативная генерация гипотез...")
                    result = enhanced_generator.generate_hypotheses(current_term, count=5, temperature=0.9)
                    if result:
                        enhanced_generator.display_hypotheses(result)

                elif action == '5':
                    print("⏳ Консервативная генерация гипотез...")
                    result = enhanced_generator.generate_hypotheses(current_term, count=5, temperature=0.3)
                    if result:
                        enhanced_generator.display_hypotheses(result)

                elif action == '6':
                    print("🎛️ Кастомная настройка генерации:")

                    # Количество гипотез
                    count_input = input("   📊 Количество гипотез (1-10) [5]: ").strip()
                    try:
                        count = int(count_input) if count_input else 5
                        count = max(1, min(10, count))
                    except ValueError:
                        count = 5

                    # Температура
                    temp_input = input("   🌡️ Температура (0.1-1.0) [0.7]: ").strip()
                    try:
                        temperature = float(temp_input) if temp_input else 0.7
                        temperature = max(0.1, min(1.0, temperature))
                    except ValueError:
                        temperature = 0.7

                    print(f"⏳ Генерация {count} гипотез с температурой {temperature}...")
                    result = enhanced_generator.generate_hypotheses(current_term, count=count, temperature=temperature)
                    if result:
                        enhanced_generator.display_hypotheses(result)

                elif action == '7':
                    current_term = None
                    continue

                elif action == '8':
                    break

                else:
                    print("⚠️ Неверный выбор")

        except KeyboardInterrupt:
            print("\n⏹️ Прерывание (Ctrl+C)")
            break
        except Exception as e:
            print(f"❌ Ошибка: {e}")
            continue
    """
    Режим без фильтрации - работа с любыми терминами
    """
    print("🚀 РЕЖИМ БЕЗ ФИЛЬТРАЦИИ - РАБОТА С ЛЮБЫМИ ТЕРМИНАМИ")
    print("=" * 70)
    print("В этом режиме вы можете:")
    print("• Ввести любой термин из графа")
    print("• Получить анализ его связей")
    print("• Сгенерировать гипотезы на основе его соседей")
    print("• Не ограничиваться ключевыми словами")
    print("=" * 70)

    # Создаем простые компоненты без фильтрации
    try:
        from openai import OpenAI
        client = OpenAI(base_url="http://80.209.242.40:8000/v1", api_key="dummy-key")

        class TempScientificGenerator:
            def __init__(self, G, node_meta, edge_meta, client):
                self.G = G
                self.node_full_meta = node_meta
                self.edge_full_meta = edge_meta
                self.client = client

        scientific_generator = TempScientificGenerator(G, node_full_meta, edge_full_meta, client)
        optimized_generator = OptimizedHypothesisGenerator(scientific_generator)

    except Exception as e:
        print(f"❌ Ошибка создания компонентов: {e}")
        return

    current_term = None

    while True:
        try:
            if current_term is None:
                print(f"\n{'🎯' * 20}")
                print("ВЫБОР ЛЮБОГО ТЕРМИНА ИЗ ГРАФА")
                print(f"{'🎯' * 20}")

                print("Варианты:")
                print("1. 🔍 Исследовать граф (показать популярные термины)")
                print("2. ✍️ Ввести термин вручную")
                print("3. 🎲 Случайный популярный термин")
                print("4. ❌ Выход")

                choice = input("\n👆 Выберите действие (1-4): ").strip()

                if choice == '1':
                    explore_graph_terms(G, node_full_meta, 30)
                    continue

                elif choice == '2':
                    manual_term = input("✍️ Введите термин: ").strip()
                    if manual_term and manual_term in G:
                        current_term = manual_term
                        degree = G.degree(manual_term)
                        neighbors_count = len(list(G.neighbors(manual_term)))
                        doi_status = "📚" if _has_doi(node_full_meta.get(manual_term, {})) else "❌"

                        print(f"\n✅ ТЕРМИН НАЙДЕН: {current_term}")
                        print(f"   🔗 Связей: {degree}")
                        print(f"   👥 Соседей: {neighbors_count}")
                        print(f"   📚 DOI: {doi_status}")
                    else:
                        print(f"❌ Термин '{manual_term}' не найден в графе")
                    continue

                elif choice == '3':
                    degree_cache = dict(G.degree())
                    popular_terms = [(term, degree) for term, degree in degree_cache.items() if degree >= 10]
                    if popular_terms:
                        current_term = random.choice(popular_terms)[0]
                        degree = degree_cache[current_term]
                        doi_status = "📚" if _has_doi(node_full_meta.get(current_term, {})) else "❌"

                        print(f"🎲 Выбран случайный популярный термин: {current_term}")
                        print(f"   🔗 Связей: {degree}")
                        print(f"   📚 DOI: {doi_status}")
                    else:
                        print("❌ Популярные термины не найдены")
                    continue

                elif choice == '4':
                    break

                else:
                    print("⚠️ Неверный выбор")
                    continue

            else:
                # Работа с выбранным термином
                print(f"\n{'🔬' * 30}")
                print(f"АНАЛИЗ ТЕРМИНА: {current_term.upper()}")
                print(f"{'🔬' * 30}")

                print("Выберите действие:")
                print("1. 🔍 Анализ всех связей")
                print("2. 💡 Генерация гипотез")
                print("3. 🔄 Выбрать новый термин")
                print("4. ❌ Выход")

                action = input("\n👆 Выберите действие (1-4): ").strip()

                if action == '1':
                    print("⏳ Анализ связей...")
                    spec_data = optimized_generator.analyze_specialized_connections(current_term)

                    if spec_data:
                        print(f"✅ Найдено {len(spec_data['specialized_neighbors'])} соседей для анализа")
                        print(f"📚 DOI доступны для {len(spec_data['doi_metadata'])} терминов")

                elif action == '2':
                    print("⏳ Генерация гипотез...")

                    # Запрашиваем температуру
                    temp_input = input("🌡️ Выберите температуру (0.1-1.0) [0.75]: ").strip()
                    try:
                        temperature = float(temp_input) if temp_input else 0.75
                        temperature = max(0.1, min(1.0, temperature))
                    except ValueError:
                        temperature = 0.75

                    spec_data = optimized_generator.analyze_specialized_connections(current_term)

                    if spec_data:
                        hypotheses = optimized_generator.generate_specialized_hypotheses(
                            current_term, spec_data, 5, temperature
                        )

                        if hypotheses:
                            optimized_generator.display_specialized_hypotheses(hypotheses)

                elif action == '3':
                    current_term = None
                    continue

                elif action == '4':
                    break

                else:
                    print("⚠️ Неверный выбор")

        except KeyboardInterrupt:
            print("\n⏹️ Прерывание (Ctrl+C)")
            break
        except Exception as e:
            print(f"❌ Ошибка: {e}")
            continue
    """
    Диагностика структуры метаданных для отладки DOI
    """
    print("🔍 ДИАГНОСТИКА СТРУКТУРЫ МЕТАДАННЫХ")
    print("=" * 60)

    if not node_full_meta:
        print("❌ node_full_meta пуст")
        return

    sample_keys = list(node_full_meta.keys())[:max_examples]

    print(f"📊 Всего терминов в метаданных: {len(node_full_meta)}")
    print(f"🔍 Анализ первых {len(sample_keys)} примеров:")

    doi_counts = {'dois': 0, 'doi': 0, 'DOI': 0, 'other': 0}

    for i, key in enumerate(sample_keys, 1):
        meta = node_full_meta[key]
        print(f"\n{i}. Термин: '{key}'")
        print(f"   Тип метаданных: {type(meta)}")

        if isinstance(meta, dict):
            print(f"   Ключи: {list(meta.keys())}")

            # Проверяем DOI поля
            for field in ['dois', 'doi', 'DOI']:
                if field in meta:
                    value = meta[field]
                    print(f"   {field}: {type(value)} = {value}")
                    if field == 'dois' and isinstance(value, list):
                        valid_dois = [d for d in value if d and str(d).strip() and str(d).strip().lower() not in ['none', 'nan', 'null']]
                        print(f"   Valid DOIs in list: {len(valid_dois)}")
                        if valid_dois:
                            doi_counts['dois'] += 1
                    elif field in ['doi', 'DOI'] and value:
                        doi_counts[field] += 1
        else:
            print(f"   Содержимое: {str(meta)[:100]}...")

    print(f"\n📊 СТАТИСТИКА DOI:")
    print(f"   Термины с 'dois' (список): {doi_counts['dois']}")
    print(f"   Термины с 'doi': {doi_counts['doi']}")
    print(f"   Термины с 'DOI': {doi_counts['DOI']}")

    # Общая статистика
    total_with_dois = 0
    all_doi_fields = ['dois', 'doi', 'DOI', 'Doi', 'pmid', 'PMID']

    for term, meta in node_full_meta.items():
        if isinstance(meta, dict):
            has_doi = False
            for field in all_doi_fields:
                if field in meta:
                    value = meta[field]
                    if isinstance(value, list):
                        if any(d and str(d).strip() and str(d).strip().lower() not in ['none', 'nan', 'null'] for d in value):
                            has_doi = True
                            break
                    elif value and str(value).strip() and str(value).strip().lower() not in ['none', 'nan', 'null']:
                        has_doi = True
                        break
            if has_doi:
                total_with_dois += 1

    print(f"\n🎯 ИТОГО: {total_with_dois} из {len(node_full_meta)} терминов имеют DOI")
    percentage = (total_with_dois / len(node_full_meta)) * 100 if node_full_meta else 0
    print(f"   Процент покрытия DOI: {percentage:.1f}%")

    if total_with_dois == 0:
        print(f"\n⚠️ РЕКОМЕНДАЦИИ:")
        print(f"   1. Проверьте, что DataFrame содержит колонку 'doi'")
        print(f"   2. Убедитесь, что build_graph_with_metadata() корректно сохраняет DOI")
        print(f"   3. Проверьте, что DOI не равны None/NaN/пустой строке")

# === ГОТОВЫЕ КОМАНДЫ ===
print("""
🎯 ЗАЩИЩЕННАЯ СПЕЦИАЛИЗИРОВАННАЯ СИСТЕМА ГОТОВА!

Основные команды:
specialized_interactive_mode(G, node_full_meta, edge_full_meta)  # С вашими переменными
diagnose_metadata_structure(node_full_meta)  # Диагностика DOI

🔧 ИСПРАВЛЕНА СТРУКТУРА DOI:
✅ Поддержка поля 'dois' как список (ваша структура)
✅ Поддержка поля 'doi' как строка
✅ Извлечение первого валидного DOI из списка
✅ Обработка titles, authors, years как списков
✅ Расширенная диагностика метаданных

🛡️ ЗАЩИТА ОТ СЛУЧАЙНЫХ ГИПОТЕЗ:
✅ Проверка качества данных перед генерацией
✅ Блокировка при отсутствии специализированных соседей
✅ Предупреждения при отсутствии DOI
✅ Запрос подтверждения при низком качестве
✅ Рекомендации по улучшению

🔍 ДИАГНОСТИКА:
Если DOI по-прежнему = 0, запустите:
diagnose_metadata_structure(node_full_meta)

ЗАПУСК:
specialized_interactive_mode(G, node_full_meta, edge_full_meta)
""")

# Функция для быстрого запуска
def quick_start():
    """Быстрый запуск с автопоиском графа"""
    print("🚀 Попытка автоматического запуска...")
    specialized_interactive_mode()


🎯 ЗАЩИЩЕННАЯ СПЕЦИАЛИЗИРОВАННАЯ СИСТЕМА ГОТОВА!

Основные команды:
specialized_interactive_mode(G, node_full_meta, edge_full_meta)  # С вашими переменными
diagnose_metadata_structure(node_full_meta)  # Диагностика DOI

🔧 ИСПРАВЛЕНА СТРУКТУРА DOI:
✅ Поддержка поля 'dois' как список (ваша структура)
✅ Поддержка поля 'doi' как строка  
✅ Извлечение первого валидного DOI из списка
✅ Обработка titles, authors, years как списков
✅ Расширенная диагностика метаданных

🛡️ ЗАЩИТА ОТ СЛУЧАЙНЫХ ГИПОТЕЗ:
✅ Проверка качества данных перед генерацией
✅ Блокировка при отсутствии специализированных соседей
✅ Предупреждения при отсутствии DOI
✅ Запрос подтверждения при низком качестве
✅ Рекомендации по улучшению

🔍 ДИАГНОСТИКА:
Если DOI по-прежнему = 0, запустите:
diagnose_metadata_structure(node_full_meta)

ЗАПУСК:
specialized_interactive_mode(G, node_full_meta, edge_full_meta)



In [69]:
specialized_interactive_mode(G, node_full_meta, edge_full_meta)

🚀 СПЕЦИАЛИЗИРОВАННЫЙ РЕЖИМ НА ОСНОВЕ ВАШИХ КЛЮЧЕВЫХ СЛОВ
Специализация:
  🧬 Биомаркеры старения человека
  🔬 Эпигенетические часы
  ⚡ Воспаление при старении
  🧪 Специфичные гены (ATAD3A, COL18A1, GATA4, etc.)
  🏥 Терапевтические мишени
  🧠 Иммунное старение
  🔀 Пути долголетия
  📊 Одноклеточный анализ
📊 Используем граф: 32,707 узлов, 775,715 рёбер
🔍 Инициализация для графа: 32,707 узлов, 775,715 рёбер
📋 Извлечено 68 уникальных ключевых слов
🚀 Построение индексов...
🔍 Поиск специализированных терминов по вашим ключевым словам...
📚 Построение индекса DOI...
🔍 Примеры метаданных для отладки:
   muscle biopsies: <class 'dict'>
      ключи: ['dois', 'years', 'titles', 'authors', 'keywords']
      dois: ['https://doi.org/10.1186/s13072-025-00601-w', nan]
   human myoblast: <class 'dict'>
      ключи: ['dois', 'years', 'titles', 'authors', 'keywords']
      dois: ['https://doi.org/10.1186/s13072-025-00601-w', 'https://doi.org/10.3390/ijms241713181']
   tet1-2: <class 'dict'>
      ключи: ['d